In [55]:
from EEG2CodeKeras import (basearchi,
                           basearchitest_batchnorm,
                           basearchi_patchembedding,
                           basearchi_patchembeddingdilation,
                           trueVanilliaEEG2Code,
                           vanilliaEEG2Code,
                           vanilliaEEG2Code2,
                           EEGnet_Inception)
from _utils import make_preds_accumul_aggresive, make_preds_pvalue


from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.pipeline import make_pipeline
from pyriemann.estimation import XdawnCovariances, Xdawn
from pyriemann.tangentspace import TangentSpace
from pyriemann.classification import MDM
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from tensorflow import keras
import mne
from mne.decoding import Vectorizer
from mne.decoding import CSP
import os
import pandas as pd

import sys
import numpy as np
from imblearn.under_sampling import RandomUnderSampler
from sklearn.metrics import accuracy_score, balanced_accuracy_score, confusion_matrix
from sklearn.utils import shuffle
np.set_printoptions(threshold=sys.maxsize)
import matplotlib.pyplot as plt
import time
from utils import balance

import torch
from torch.utils.data import DataLoader, TensorDataset
from SPDNet.torch.spd_net_bn_torch import SPDNetBN_Module
from SPDNet.torch.optimizers import riemannian_adam as torch_riemannian_adam

from Alignments.riemannian import compute_riemannian_alignment
from utils import Euc2SPD
from SPDNet.tensorflow.spd_net_tensorflow import SPDNet_Tensorflow



c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\moabb\pipelines\__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


In [56]:
import tensorflow as tf
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [57]:
fps = 60
sfreq = 500


## Path to the data

In [3]:
participants = ['P1','P2','P3','P4','P5','P6','P7','P8','P9','P10',
                'P11','P12','P13','P14','P15','P16','P17','P18','P19','P20',
                'P21','P22','P23','P24']
path = 'C:\\Users\\s.velut\\Documents\\These\\Protheus_PHD\\Data\\Dry_Ricker'
# path = '/home/dcas/k.cabrera/Data/SETNOP2'
n_class=5
window_size = 0.25



# file_name = '_'.join([participant, 'mseq40.set'])
# file_name = '_'.join([participant, 'mseq100.set'])
# file_name = '_'.join([participant, 'burst40.set'])
# file_name = '_'.join([participant, 'dryburst100.set'])
# file_name = '/'.join([path,  participant+'_whitemseq.set'])
# file_name = '_'.join([participant, 'burst', 'oi_1.set'])


## Load the raw data and small pre-process
1. Drop the ACC channels and the shitty channels near ears
2. Average re-referencing
4. Extract 2.2s epochs using events

In [4]:
%matplotlib Qt

In [61]:
sfreq = 500
raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
               for i in range(len(participants))]

Sampling frequency of the instance is already 500.0, returning unmodified.
Sampling frequency of the instance is already 500.0, returning unmodified.


C:\Users\s.velut\AppData\Local\Temp\ipykernel_9712\4146463880.py:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Users\s.velut\AppData\Local\Temp\ipykernel_9712\4146463880.py:2: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Users\s.velut\AppData\Local\Temp\ipykernel_9712\4146463880.py:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Users\

Sampling frequency of the instance is already 500.0, returning unmodified.
Sampling frequency of the instance is already 500.0, returning unmodified.
Sampling frequency of the instance is already 500.0, returning unmodified.


C:\Users\s.velut\AppData\Local\Temp\ipykernel_9712\4146463880.py:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Users\s.velut\AppData\Local\Temp\ipykernel_9712\4146463880.py:2: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Users\s.velut\AppData\Local\Temp\ipykernel_9712\4146463880.py:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Users\

Sampling frequency of the instance is already 500.0, returning unmodified.
Sampling frequency of the instance is already 500.0, returning unmodified.
Sampling frequency of the instance is already 500.0, returning unmodified.
Sampling frequency of the instance is already 500.0, returning unmodified.


...

C:\Users\s.velut\AppData\Local\Temp\ipykernel_9712\4146463880.py:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Users\s.velut\AppData\Local\Temp\ipykernel_9712\4146463880.py:2: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Users\s.velut\AppData\Local\Temp\ipykernel_9712\4146463880.py:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Users\

Sampling frequency of the instance is already 500.0, returning unmodified.
Sampling frequency of the instance is already 500.0, returning unmodified.
Sampling frequency of the instance is already 500.0, returning unmodified.


C:\Users\s.velut\AppData\Local\Temp\ipykernel_9712\4146463880.py:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Users\s.velut\AppData\Local\Temp\ipykernel_9712\4146463880.py:2: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Users\s.velut\AppData\Local\Temp\ipykernel_9712\4146463880.py:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Users\

Sampling frequency of the instance is already 500.0, returning unmodified.
Sampling frequency of the instance is already 500.0, returning unmodified.
Sampling frequency of the instance is already 500.0, returning unmodified.


C:\Users\s.velut\AppData\Local\Temp\ipykernel_9712\4146463880.py:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Users\s.velut\AppData\Local\Temp\ipykernel_9712\4146463880.py:2: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Users\s.velut\AppData\Local\Temp\ipykernel_9712\4146463880.py:2: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw_eeglab = [mne.io.read_raw_eeglab(os.path.join(path, '_'.join([participants[i], 'dryburst100.set'])), preload=True, verbose=False).resample(sfreq=sfreq)
C:\Users\

In [62]:
# to_drop = ["P9", "P10", "TP9", "TP10", "10", "21"]
# keep = ["O1", "O2", "Oz", "P7", "P3", "P4", "P8", "Pz"]
# keep = ["16", "18", "17", "15", "14", "19", "20", "13"] # electrodes to keep

for ind_i,i in enumerate(participants):
    # raw = raw.drop_channels([ch for ch in raw.ch_names if ch in to_drop])
    # raw = raw.drop_channels([i for i in raw.ch_names if i not in keep])

    raw_eeglab[ind_i] = raw_eeglab[ind_i].filter(l_freq=1, h_freq=25, method="fir", verbose=True)

    # mne.set_eeg_reference(raw_eeglab[ind_i], 'average', copy=False, verbose=False)
    n_channels = len(raw_eeglab[ind_i].ch_names)
    print("Channels :", raw_eeglab[ind_i].ch_names)
    # print(raw_eeglab[ind_i].info)



# raw = raw.filter(l_freq=50.1, h_freq=49.9, method="iir", verbose=True)
# raw = mne.filter.resample(raw,2)
# raw.resample(480, npad='auto')
# Average re-referencing
#raw = raw.filter(l_freq=5, h_freq=45, method="fir", verbose=True)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (3.302 s)



Channels : ['EEG 000', 'EEG 001', 'EEG 002', 'EEG 003', 'EEG 004', 'EEG 005', 'EEG 006', 'EEG 007']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 25.00 Hz
- Upper transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 28.12 Hz)
- Filter length: 1651 samples (3.302 s)

Channels : ['EEG 000', 'EEG 001', 'EEG 002', 'EEG 003', 'EEG 004', 'EEG 005', 'EEG 006', 'EEG 007']
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 25 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method


In [16]:
p=1
window_size=0.5
events, event_id = mne.events_from_annotations(raw_eeglab[p])
print(event_id,events)
epochs = mne.Epochs(raw_eeglab[p], events, event_id=event_id, tmin=0, \
            tmax=0.6, baseline=(None, None), preload=False, verbose=False)

epochs_train1 = mne.Epochs(raw_eeglab[p],events, 6,tmin=-0.01,tmax=window_size)
epochs_train2 = mne.Epochs(raw_eeglab[p],events, 7,tmin=-0.01,tmax=window_size)
epochs_train3 = mne.Epochs(raw_eeglab[p],events, 8,tmin=-0.01,tmax=window_size)
epochs_train4 = mne.Epochs(raw_eeglab[p],events, 9,tmin=-0.01,tmax=window_size)
epochs_train5 = mne.Epochs(raw_eeglab[p],events, 10,tmin=-0.01,tmax=window_size)


Used Annotations descriptions: ['000000000000000000001000000000000000000000100000000000000000000010000000000000000000001000000000000000000000100000000000000000000010_1', '000000000000000010000000000000000000001000000000000000000000100000000000000000000010000000000000000000001000000000000000000000100000_3', '000000000010000000000000000000001000000000000000000000100000000000000000000010000000000000000000001000000000000000000000100000000000_2', '000000001000000000000000000000100000000000000000000010000000000000000000001000000000000000000000100000000000000000000010000000000000_5', '100000000000000000000010000000000000000000001000000000000000000000100000000000000000000010000000000000000000001000000000000000000000_4', 'BURST_1', 'BURST_2', 'BURST_3', 'BURST_4', 'BURST_5', 'boundary']
{'000000000000000000001000000000000000000000100000000000000000000010000000000000000000001000000000000000000000100000000000000000000010_1': 1, '00000000000000001000000000000000000000100000000000000000000010000000

In [9]:
evoked_1 = list(epochs_train1.iter_evoked())
evoked_2 = list(epochs_train2.iter_evoked())
evoked_3 = list(epochs_train3.iter_evoked())
evoked_4 = list(epochs_train4.iter_evoked())
evoked_5 = list(epochs_train5.iter_evoked())

evokeds = dict(one=evoked_1,two=evoked_2,three=evoked_3,four=evoked_4,five=evoked_5)
mne.viz.plot_compare_evokeds(evokeds, combine="mean", picks="EEG 000")

C:\Users\s.velut\AppData\Local\Temp\ipykernel_20788\3746502456.py:8: RuntimeWarning: Only 1 channel in "picks"; cannot combine by method "mean".
  mne.viz.plot_compare_evokeds(evokeds, combine="mean", picks="EEG 000")


combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"


C:\Users\s.velut\AppData\Local\Temp\ipykernel_20788\3746502456.py:8: RuntimeWarning: Cannot find channel coordinates in the supplied Evokeds. Not showing channel locations.
  mne.viz.plot_compare_evokeds(evokeds, combine="mean", picks="EEG 000")


[<Figure size 1000x750 with 1 Axes>]

In [63]:
epochs_list = []
events_list = []
events_id_list = []
onset_code_list = []
data_list = []
labels_code_list = []
for ind_i, i in enumerate(participants): 
    # Strip the annotations that were script to make them easier to process
    events, event_id = mne.events_from_annotations(raw_eeglab[ind_i], event_id='auto', verbose=False)
    to_remove = []
    for idx in range(len(raw_eeglab[ind_i].annotations.description)):
        if (('boundary' in raw_eeglab[ind_i].annotations.description[idx]) or
            ('BURST' in raw_eeglab[ind_i].annotations.description[idx])):
            to_remove.append(idx)

    to_remove = np.array(to_remove)
    if len(to_remove) > 0:
        raw_eeglab[ind_i].annotations.delete(to_remove)
    # Get the events
    temp_event,temp_event_id = mne.events_from_annotations(raw_eeglab[ind_i], event_id='auto', verbose=False)
    events_list.append(temp_event)
    events_id_list.append(temp_event_id)
    shift = 0.0
    # Epoch the data following event
    epochs_list.append(mne.Epochs(raw_eeglab[ind_i], events_list[ind_i], event_id=events_id_list[ind_i], tmin=shift, \
                tmax=2.2+shift, baseline=(None, None), preload=False, verbose=False))
    # print(events_list[ind_i])

    labels_code_list.append(epochs_list[ind_i].events[..., -1])
    labels_code_list[ind_i] -= np.min(labels_code_list[ind_i])
    # print(epochs.events[..., -1])
    data_list.append(epochs_list[ind_i].get_data())
    info_ep = epochs_list[ind_i].info
    # print(epochs)
    onset_code_list.append(epochs_list[ind_i].events[..., 0])
data_list = np.array(data_list)

Using data from preloaded Raw for 75 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 75 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 75 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 75 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 75 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 75 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 75 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 75 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 75 events and 1101 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 75 events and 1101 original time points ...
0 bad epochs dropped
Using data

### Transform a code in `str` to a code in np.array

In [64]:
def code2array(code):
    tmp = []
    for idx, c in enumerate(code[:-2]):
        if c == '5' or c == '.':
            continue
        elif c == '0':
            if code[idx+2] == '5':
                tmp.append(0.5)
            else:
                tmp.append(0)
        else:
            tmp.append(1)
    if code[-1] == '.':
        if code[-2] == '0':
            tmp.append(0)
        else:
            tmp.append(1)
    return np.array(tmp)

### Build a dictionnary that contains all the code in the np.array format

In [65]:
from collections import OrderedDict
codes = OrderedDict()
for k, v in events_id_list[0].items():
    code = k.split('_')[0]
    code = code.replace('.','').replace('2','')
    idx = k.split('_')[1]
    codes[v-1] = np.array(list(map(int, code)))


### Define train/test split and windows size
Here we use only the first 7 blocks as calibrition and 8 others would be used as testing

In [66]:
np.array(codes).shape

()

In [67]:
print(data_list.shape)

(24, 75, 8, 1101)


### Slice the epoch in windows
The network is not processing full epochs but windows of 250ms. So each epoch is cut into window and the following code (`0` or `1`) is associated as label.

In [68]:
def to_window_cov(data, labels,length,n_samples_windows,codes,normalise=False,window_size=0.25,sfreq=500,fps=60,n_channels=8):
    X = np.empty(shape=((length)*data.shape[0], data.shape[1], n_samples_windows))
    y = np.empty(shape=((length)*data.shape[0]), dtype=int)
    count = 0
    for trial_nb, trial in enumerate(data):
        lab = labels[trial_nb]
        c = codes[lab]
        code_pos = 0
        for idx in range(length):
            X[count] = trial[:, idx:idx+n_samples_windows]
            if idx/sfreq >= (code_pos+1)/fps:
                code_pos += 1 
            y[count] = int(c[code_pos])
            count += 1
    if normalise:
        X_std = X.std(axis=0)
        X /= X_std + 1e-8
    xdawncov = XdawnCovariances(estimator="lwf",xdawn_estimator="lwf",nfilter=8)
    X = xdawncov.fit_transform(X,y)
    # X = X.astype(np.float32)
    y_pred = np.vstack((y,np.abs(1-y))).T
    y = np.array([1 if (y >= 0.5) else 0 for y in y_pred[:,0]])
    return X, y

def to_window_old(data, labels,length,n_samples_windows,codes,window_size=0.25,normalise=True,sfreq=500,fps=60,n_channels=8):
    
    X = np.empty(shape=((length)*data.shape[0], n_channels, n_samples_windows))
    idx_taken = []
    y = np.empty(shape=((length)*data.shape[0]), dtype=int)
    count = 0
    for trial_nb, trial in enumerate(data):
        lab = labels[trial_nb]
        c = codes[lab]
        code_pos = 0
        for idx in range(length):
            X[count] = trial[:, idx:idx+n_samples_windows]
            if idx/sfreq >= (code_pos+1)/fps:
                code_pos += 1 
            y[count] = int(c[code_pos])
            count += 1
        
        for idx in range(length):
            # print('Xidx:', trial_nb*length+idx, "Tidxm:", idx, 'TidxM:', idx +
            #       n_samples_windows, 'Ltrial', trial[:, idx:idx+n_samples_windows].shape)
            idx_taken.append(trial_nb*length+idx)
    # if normalise:
    #     X_std = X.std(axis=0)
    #     X /= X_std + 1e-8
    y_pred = np.vstack((y,np.abs(1-y))).T
    y = np.array([1 if (y >= 0.5) else 0 for y in y_pred[:,0]])

    return X, y, np.array(idx_taken)

In [80]:
window_size = 0.25
n_samples_windows = int(window_size*sfreq)
# length = int((2.2-window_size)*sfreq)
length = int((2.2-window_size)*fps)
X = np.zeros((data_list.shape[0],length*data_list.shape[1],data_list.shape[2],n_samples_windows))
y = np.zeros((data_list.shape[0],length*data_list.shape[1]))
idx_taken = np.zeros((data_list.shape[0],length*data_list.shape[1]))
domains = []
print(y.shape)
print(X.shape)
for ind_i,i in enumerate(participants):
    # X[ind_i],y[ind_i],idx_taken[ind_i] = to_window_old(data_list[ind_i],labels_code_list[ind_i],length,n_samples_windows,codes,window_size=window_size,sfreq=sfreq)
    X[ind_i],y[ind_i],idx_taken[ind_i] = to_window_old(data_list[ind_i],labels_code_list[ind_i],length,n_samples_windows,codes,window_size=window_size,sfreq=fps)
    domains.append(["Source_sub_{}".format(ind_i+1),]*len(y[ind_i]))

# print(codes[labels_test[0]])
print(y.shape)
print(X.shape)
domains = np.array(domains)

(24, 8775)
(24, 8775, 8, 125)
(24, 8775)
(24, 8775, 8, 125)


In [77]:
def onset_anno(onset_window,label_window,onset_code,nb_seq_min,nb_seq_max,code_freq,sfreq,win_size):
    assert(sfreq!=0)
    new_onset = []
    new_onset_0 = []
    current_code = 0
    onset_code = np.ceil(onset_code*code_freq/sfreq)
    nb_seq_min-=1
    onset_shift = onset_code[current_code+nb_seq_min]
    time_trial = (2.2-win_size)
    # onset_window = np.arange(0,time_trial*code_freq*(nb_seq_max-nb_seq_min)-1,1,dtype=int)
    for i,o in enumerate(onset_window):
        if label_window[i]==1:
            # print(i)
            if current_code==nb_seq_max-1-nb_seq_min:
                new_onset.append(o+onset_shift)
            else:
                if o+onset_shift >= onset_code[current_code+nb_seq_min]+time_trial*code_freq:
                    current_code+=1
                    onset_shift = onset_code[current_code+nb_seq_min]-time_trial*code_freq*current_code
                new_onset.append(o+onset_shift)
        else:
            if current_code==nb_seq_max-1-nb_seq_min:
                new_onset_0.append(o+onset_shift)
            else:
                if o+onset_shift >= onset_code[current_code+nb_seq_min]+time_trial*code_freq:
                    current_code+=1
                    onset_shift = onset_code[current_code+nb_seq_min]-time_trial*code_freq*current_code
                new_onset_0.append(o+onset_shift)
    
    # modified_onset_code = [onset_code[i]-time_trial*sfreq*i for i in range(nb_seq_min,nb_seq_max)]
    # new_onset_0 = np.concatenate([np.arange(onset_code[i],onset_code[i]+time_trial*sfreq,sfreq//60) for i in range(nb_seq_min,nb_seq_max)])
    new_onset_0 = np.array(list(filter(lambda i: i not in new_onset, new_onset_0)))
    # print(new_onset_0.shape)
    return np.array(new_onset)/code_freq, np.array(new_onset_0)/code_freq
            

In [82]:
# print(Y_train)
# print(np.where(Y_train==1)[0]*sfreq//60)
# print(frame_train)
for ind_i,i in enumerate(participants):
    onset,onset_0 = onset_anno(idx_taken[ind_i],y[ind_i],onset_code_list[ind_i],1,n_class*15,60,500,window_size)
    # print(onset,onset_0)
    anno = mne.Annotations(onset,0.001*sfreq//60,"1")
    anno.append(onset_0,0.001*sfreq//60,"0")

    raw_eeglab[ind_i] = raw_eeglab[ind_i].set_annotations(anno)
# events, event_id = mne.events_from_annotations(raw)
# print(event_id,events)
# epochs = mne.Epochs(raw, events, event_id=event_id, tmin=0, \
#             tmax=0.6, baseline=(None, None), preload=False, verbose=False)

# epochs_train1 = mne.Epochs(raw,events, 2,tmin=-0.01,tmax=window_size)
# epochs_train0 = mne.Epochs(raw,events, 1,tmin=-0.01,tmax=window_size)

# evoked_1 = list(epochs_train1.iter_evoked())
# # evoked_1.plot(picks="Oz")
# evoked_0 = list(epochs_train0.iter_evoked())
# # evoked_0.plot(picks="Oz")
# evokeds = dict(one=evoked_1,zero=evoked_0)
# mne.viz.plot_compare_evokeds(evokeds, combine="mean", picks="EEG 001")
# raw.plot(n_channels=15, duration=300)

In [17]:
p = 5
plt.plot(np.mean(X[p][np.where(Y[p]==1)],axis=0),color='r')
plt.plot(np.mean(X[p][np.where(Y[p]==0)],axis=0),color='g')
plt.show()

In [29]:
window_size

0.25

In [72]:
def mpaa(y_pred, codes, min_len=30, sfreq=60, consecutive=30, window_size=0.25):
    length = int((2.2-window_size)*sfreq)
    y_pred = np.array(y_pred)
    rez_acc = []

    code_buffer = []
    labels_pred = []
    code_pos = 0
    y_tmp = [] 
    mean_long = []

    for trial in range(int(len(y_pred)/length)):   
        # Retrieve a trial
        tmp_code = y_pred[trial*length:(trial+1)*length]
        code_pos = 0

        # Do an average over the prdata, codes, labels, fps
        code_buffer = []
        for idx in range(len(tmp_code)):
            y_tmp.append(tmp_code[idx])
            if (idx+1)/sfreq >= (code_pos+1)/60:
                code_pred = np.mean(y_tmp) 
                code_pred = int(np.rint(code_pred))
                code_buffer.append(code_pred) 
                y_tmp = []
                code_pos += 1
        # Find the code that correlate the most
        corr = -2
        pred_lab = -1
        # print("buffer:",code_buffer)
        out = 0
        for long in np.arange(min_len, len(code_buffer) -1 , step=1):
            dtw_values = []
            for key, values in codes.items():
                dtw_values.append(np.corrcoef(code_buffer[:long], values[:long])[0,1])
                # print((code_buffer[:long], values[:long]))
            dtw_values = np.array(dtw_values)
            # print("corr values;",dtw_values)
            max_dtw = list(codes.keys())[np.argmax(dtw_values)] 
            # print(max_dtw)
            if (max_dtw == pred_lab):
                out += 1
                corr = np.max(dtw_values)
            else:
                pred_lab = max_dtw
                out = 0
            if out == consecutive:
                mean_long.append((long)/60)
                break
        labels_pred.append(pred_lab)
    labels_pred = np.array(labels_pred)
    return labels_pred, code_buffer, mean_long

# Test on new dataset

## CNN

In [20]:
def expand_dim(X):
    X_temp = []
    Y_temp = []
    for i in range(X.shape[0]):
        # print(i)
        X_temp.append(np.expand_dims(X[i],1))
    return np.array(X_temp)


In [26]:
X = np.zeros((X.shape[0],length*data_list.shape[1],data_list.shape[2],int(window_size*sfreq)))
y = np.zeros((data_list.shape[0],length*data_list.shape[1]))
for ind_i,i in enumerate(participants):
    events, event_id = mne.events_from_annotations(raw_eeglab[ind_i])
    # print(event_id)
    epochs = mne.Epochs(raw_eeglab[ind_i],events,event_id,tmin=0.0,tmax=window_size,baseline=(0,0))
    temp_X = epochs.get_data()[:,:,:-1]
    y[ind_i] = epochs.events[...,-1]-1
    X[ind_i] = compute_riemannian_alignment(temp_X, mean=None, dtype='real')
    # X[ind_i] = temp_X


Used Annotations descriptions: ['0', '1']
Not setting metadata
8775 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 8775 events and 126 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['0', '1']
Not setting metadata
8775 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 8775 events and 126 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['0', '1']
Not setting metadata
8775 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 8775 events and 126 original time points ...
0 bad epochs dropped
Used Annotations descriptions: ['0', '1']
Not setting metadata
8775 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 8775 events and 126 or

### CNN SS

In [22]:
n_cal = 7
n_class = 5
nb_part = len(participants)
CNN_accuracy_code_perso = np.zeros(nb_part)
CNN_tps_train_code_perso = np.zeros(nb_part)
CNN_tps_test_code_perso = np.zeros(nb_part)
CNN_accuracy_perso = np.zeros(nb_part)
nb_samples_windows = int((2.2-window_size)*n_class*n_cal*60)
history_list = []


for i in range(nb_part):
    print("TL to the participant : ", i)

    X_train = np.expand_dims(X[i][:nb_samples_windows],1)
    Y_train = y[i][:nb_samples_windows]
    # Y_train = np.vstack((Y_train,np.abs(1-Y_train))).T
    domains_train = domains[i][:nb_samples_windows]
    X_test = np.expand_dims(X[i][nb_samples_windows:],1)
    Y_test = np.vstack((y[i][nb_samples_windows:],np.abs(1-y[i][nb_samples_windows:]))).T
    labels_code_test = labels_code_list[i][n_cal*n_class:]

    print(X_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train,Y_train,domains_train = balance(X_train,Y_train,domains_train)
    Y_train = np.vstack((Y_train,np.abs(1-Y_train))).T
    n_samples_windows = X_train.shape[-1]

    print("Creating the different pipelines")
    clf = basearchi(windows_size = n_samples_windows, n_channel_input = X_train.shape[2])
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    batchsize = 64 #128 # 64 for burst
    epochs = 20 #45 # 20 for burst

    print("Fitting")
    start = time.time()
    lr = 1e-3
    weight_decay = 1e-4
    optimizer = keras.optimizers.Adam(learning_rate=lr, amsgrad=True)
    clf.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
    history_list.append(clf.fit(np.array(x_train), y_train,
                    batch_size=batchsize, epochs=epochs,
                    validation_data=(np.array(x_val), y_val), shuffle=True))
    CNN_tps_train_code_perso[i] = time.time() - start

    print("getting accuracy of participant ", i)
    start = time.time()
    y_pred = clf.predict(X_test)[:,0]
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y[0] == 0 else 1 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    CNN_accuracy_perso[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    labels_pred_accumul, _, mean_long_accumul = mpaa(
        y_pred_norm, codes, min_len=30, fps=60, consecutive=50, window_size=window_size
    )
    print("meannnnnn long",np.mean(mean_long_accumul))
    CNN_tps_test_code_perso[i] = time.time() - start
    CNN_accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(CNN_accuracy_perso)
print(CNN_tps_train_code_perso)
print(CNN_tps_test_code_perso)
print(CNN_accuracy_code_perso)
# pd.DataFrame(CNN_accuracy_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/score/SS_score.csv")
# pd.DataFrame(CNN_accuracy_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/score_code/SS_score_code.csv")
# pd.DataFrame(CNN_tps_train_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/temps_train_code/SS_tps_train_code.csv")
# pd.DataFrame(CNN_tps_test_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/temps_test_code/SS_tps_test_code.csv")


TL to the participant :  0
(3990, 1, 8, 150)
(4560, 1, 8, 150)
balancing the number of ones and zeros
Creating the different pipelines


Fitting
Epoch 1/20


6/6 [==============================] - 2s 87ms/step - loss: 0.7866 - accuracy: 0.5352 - val_loss: 0.6493 - val_accuracy: 0.7027
Epoch 2/20
6/6 [==============================] - 0s 24ms/step - loss: 0.6697 - accuracy: 0.6116 - val_loss: 0.5670 - val_accuracy: 0.7568
Epoch 3/20
6/6 [==============================] - 0s 23ms/step - loss: 0.5636 - accuracy: 0.7523 - val_loss: 0.5205 - val_accuracy: 0.7568
Epoch 4/20
6/6 [==============================] - 0s 23ms/step - loss: 0.4914 - accuracy: 0.7798 - val_loss: 0.5007 - val_accuracy: 0.8108
Epoch 5/20
6/6 [==============================] - 0s 23ms/step - loss: 0.4215 - accuracy: 0.8318 - val_loss: 0.5305 - val_accuracy: 0.8108
Epoch 6/20
6/6 [==============================] - 0s 24ms/step - loss: 0.3862 - accuracy: 0.8349 - val_loss: 0.5653 - val_accuracy: 0.8108
Epoch 7/20
6/6 [===

In [7]:
print("with 0.3s",np.mean([0.98, 0.85, 0.75, 0.85, 0.95, 0.98, 0.72, 0.95, 0.75, 0.38, 0.9,  0.8,  0.82, 0.88,
 0.4,  0.95, 0.92, 0.75, 0.78, 0.55, 1,0.7,  0.7,  1  ]))
np.mean([1,0.98, 0.88, 0.85, 0.98, 0.9,  0.68, 0.92, 0.68, 0.52, 0.8,  0.65, 0.6,  0.88,
 0.32, 0.92, 0.82, 0.9,  0.72, 0.65, 0.95, 0.9,  0.6,  1])

with 0.3s 0.8045833333333334


0.7958333333333334

In [ ]:
labels_pred_accumul,labels_code_test

pred_train = clf.predict(X_train)[:,0]
y_pred_train = np.array([1 if (y >= 0.5) else 0 for y in pred_train])
y_train_norm = np.array([0 if y[0] == 0 else 1 for y in Y_train])

print(balanced_accuracy_score(y_train_norm,y_pred_train))
print(y_pred_train.shape)

labels_pred_accumul, _, mean_long_accumul = mpaa(
    y_train_norm, codes, min_len=30, fps=60, consecutive=50, window_size=window_size
)
print(labels_pred_accumul.shape)
print(np.round(accuracy_score(labels_code_list[i][:7*5][labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2))
   

128/128 [==============================] - 0s 3ms/step
0.9427549194991056
(4095,)
(35,)
1.0


In [ ]:
print((np.array(codes[1]),y_pred_train[:117]))
print((np.array(codes[0]),y_pred_train[117:117*2]))

labels_code_list[i],labels_pred_accumul

### CNN DG

In [23]:

n_cal = 4
n_class = 5
nb_part = len(participants)
CNN_accuracy_code_perso = np.zeros(nb_part)
CNN_tps_train_code_perso = np.zeros(nb_part)
CNN_tps_test_code_perso = np.zeros(nb_part)
CNN_accuracy_perso = np.zeros(nb_part)
nb_samples_windows = int((2.2-window_size)*n_class*n_cal*60)
history_list = []


for i in range(nb_part):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(nb_part) if j!=i]

    X_train = np.expand_dims(np.concatenate(X[ind2take]).reshape(-1,X.shape[-2],X.shape[-1]),1)
    Y_train = np.concatenate(y[ind2take]).reshape(-1)
    domains_train = np.concatenate(domains[ind2take]).reshape(-1)
    X_test = np.expand_dims(X[i],1)
    Y_test = np.vstack((y[i],np.abs(1-y[i]))).T
    labels_code_test = labels_code_list[i]

    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train,Y_train,domains_train = balance(X_train,Y_train,domains_train)
    Y_train = np.vstack((Y_train,np.abs(1-Y_train))).T
    n_samples_windows = X_train.shape[-1]

    print("Creating the different pipelines")
    clf = basearchi(windows_size = n_samples_windows, n_channel_input = X_train.shape[2])
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    batchsize = 64 #128 # 64 for burst
    epochs = 20 #45 # 20 for burst

    print("Fitting")
    start = time.time()
    lr = 1e-3
    weight_decay = 1e-4
    optimizer = keras.optimizers.Adam(learning_rate=lr, amsgrad=True)
    clf.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
    history_list.append(clf.fit(np.array(x_train), y_train,
                    batch_size=batchsize, epochs=epochs,
                    validation_data=(np.array(x_val), y_val), shuffle=True))
    CNN_tps_train_code_perso[i] = time.time() - start

    print("getting accuracy of participant ", i)
    start = time.time()
    y_pred = clf.predict(X_test)[:,0]
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y[0] == 0 else 1 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    CNN_accuracy_perso[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    labels_pred_accumul, _, mean_long_accumul = mpaa(
        y_pred_norm, codes, min_len=30, fps=60, consecutive=50, window_size=window_size
    )
    print("meannnnnn long",np.mean(mean_long_accumul))
    CNN_tps_test_code_perso[i] = time.time() - start
    CNN_accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(CNN_accuracy_perso)
print(CNN_tps_train_code_perso)
print(CNN_tps_test_code_perso)
print(CNN_accuracy_code_perso)
# pd.DataFrame(CNN_accuracy_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/score/DG_score.csv")
# pd.DataFrame(CNN_accuracy_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/score_code/DG_score_code.csv")
# pd.DataFrame(CNN_tps_train_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/temps_train_code/DG_tps_train_code.csv")
# pd.DataFrame(CNN_tps_test_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/temps_test_code/DG_tps_test_code.csv")



TL to the participant :  0
(196650, 1, 8, 150)
(196650,)
(8550, 1, 8, 150)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
253/253 [==============================] - 7s 17ms/step - loss: 0.6562 - accuracy: 0.6245 - val_loss: 0.5792 - val_accuracy: 0.7074
Epoch 2/20
253/253 [==============================] - 4s 16ms/step - loss: 0.5955 - accuracy: 0.6898 - val_loss: 0.5627 - val_accuracy: 0.7207
Epoch 3/20
253/253 [==============================] - 4s 16ms/step - loss: 0.5787 - accuracy: 0.7033 - val_loss: 0.5620 - val_accuracy: 0.7107
Epoch 4/20
253/253 [==============================] - 4s 16ms/step - loss: 0.5749 - accuracy: 0.7072 - val_loss: 0.5528 - val_accuracy: 0.7224
Epoch 5/20
253/253 [==============================] - 4s 16ms/step - loss: 0.5647 - accuracy: 0.7176 - val_loss: 0.5500 - val_accuracy: 0.7285
Epoch 6/20
253/253 [==============================] - 4s 15ms/step - loss: 0.5579 - accuracy: 0.7222 - val_loss: 0.5513 - val_accu

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.439236111111111
TL to the participant :  1
(196650, 1, 8, 150)
(196650,)
(8550, 1, 8, 150)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
253/253 [==============================] - 7s 18ms/step - loss: 0.6643 - accuracy: 0.6220 - val_loss: 0.5649 - val_accuracy: 0.7269
Epoch 2/20
253/253 [==============================] - 4s 17ms/step - loss: 0.6001 - accuracy: 0.6869 - val_loss: 0.5395 - val_accuracy: 0.7297
Epoch 3/20
253/253 [==============================] - 4s 16ms/step - loss: 0.5828 - accuracy: 0.7051 - val_loss: 0.5279 - val_accuracy: 0.7408
Epoch 4/20
253/253 [==============================] - 4s 16ms/step - loss: 0.5717 - accuracy: 0.7090 - val_loss: 0.5235 - val_accuracy: 0.7475
Epoch 5/20
253/253 [==============================] - 4s 15ms/step - loss: 0.5706 - accuracy: 0.7149 - val_loss: 0.5242 - val_accuracy: 0.7508
Epoch 6/20
253/253 [==============================] - 4s 16ms/step - loss: 0.5668 - accuracy: 0.7

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.433333333333333
TL to the participant :  5
(196650, 1, 8, 150)
(196650,)
(8550, 1, 8, 150)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
253/253 [==============================] - 5s 13ms/step - loss: 0.6746 - accuracy: 0.5956 - val_loss: 0.5847 - val_accuracy: 0.7124
Epoch 2/20
253/253 [==============================] - 3s 12ms/step - loss: 0.6096 - accuracy: 0.6747 - val_loss: 0.5697 - val_accuracy: 0.7191
Epoch 3/20
253/253 [==============================] - 3s 13ms/step - loss: 0.5916 - accuracy: 0.6896 - val_loss: 0.5646 - val_accuracy: 0.7263
Epoch 4/20
253/253 [==============================] - 3s 13ms/step - loss: 0.5824 - accuracy: 0.7019 - val_loss: 0.5559 - val_accuracy: 0.7380
Epoch 5/20
253/253 [==============================] - 3s 12ms/step - loss: 0.5784 - accuracy: 0.7039 - val_loss: 0.5466 - val_accuracy: 0.7352
Epoch 6/20
253/253 [==============================] - 3s 12ms/step - loss: 0.5749 - accuracy: 0.7

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.3898809523809526
TL to the participant :  18
(196650, 1, 8, 150)
(196650,)
(8550, 1, 8, 150)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
253/253 [==============================] - 5s 13ms/step - loss: 0.6764 - accuracy: 0.5948 - val_loss: 0.5797 - val_accuracy: 0.7163
Epoch 2/20
253/253 [==============================] - 3s 12ms/step - loss: 0.5898 - accuracy: 0.6931 - val_loss: 0.5344 - val_accuracy: 0.7531
Epoch 3/20
253/253 [==============================] - 3s 12ms/step - loss: 0.5710 - accuracy: 0.7137 - val_loss: 0.5299 - val_accuracy: 0.7497
Epoch 4/20
253/253 [==============================] - 3s 12ms/step - loss: 0.5624 - accuracy: 0.7189 - val_loss: 0.5271 - val_accuracy: 0.7598
Epoch 5/20
253/253 [==============================] - 3s 12ms/step - loss: 0.5546 - accuracy: 0.7248 - val_loss: 0.5187 - val_accuracy: 0.7547
Epoch 6/20
253/253 [==============================] - 3s 12ms/step - loss: 0.5490 - accuracy: 0

In [8]:
print("with 0.3s",np.mean([0.89, 0.91, 0.24, 0.95, 0.93, 0.93, 0.92, 0.93, 0.77, 0.69, 0.8,  0.65, 0.84, 0.91,
 0.53, 0.95, 0.92, 0.92, 0.13, 0.83, 0.68, 0.4,  0.49, 0.87]))
print("with 0.25s",np.mean([0.49, 0.81, 0.25, 0.93, 0.75, 0.88, 0.81, 0.92, 0.84, 0.69, 0.76, 0.63, 0.73, 0.84,
 0.47, 0.92, 0.89, 0.89, 0.2,  0.76, 0.71, 0.39, 0.41, 0.91]))

with 0.3s 0.7533333333333333
with 0.25s 0.7033333333333333


### CNN DA

In [24]:

n_cal = 4
n_class = 5
nb_part = len(participants)
CNN_accuracy_code_perso = np.zeros(nb_part)
CNN_tps_train_code_perso = np.zeros(nb_part)
CNN_tps_test_code_perso = np.zeros(nb_part)
CNN_accuracy_perso = np.zeros(nb_part)
nb_samples_windows = int((2.2-window_size)*n_class*n_cal*60)
history_list = []


for i in range(nb_part):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(nb_part) if j!=i]

    X_train = np.expand_dims(np.concatenate([X[ind2take].reshape(-1,X.shape[-2],X.shape[-1]),X[i][:nb_samples_windows]]).reshape(-1,X.shape[-2],X.shape[-1]),1)
    Y_train = np.concatenate([y[ind2take].reshape(-1),y[i][:nb_samples_windows]]).reshape(-1)
    domains_train = np.concatenate([domains[ind2take].reshape(-1),domains[i][:nb_samples_windows]]).reshape(-1)
    X_test = np.expand_dims(X[i][nb_samples_windows:],1)
    Y_test = np.vstack((y[i][nb_samples_windows:],np.abs(1-y[i][nb_samples_windows:]))).T
    labels_code_test = labels_code_list[i][n_cal*n_class:]

    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train,Y_train,domains_train = balance(X_train,Y_train,domains_train)
    Y_train = np.vstack((Y_train,np.abs(1-Y_train))).T
    n_samples_windows = X_train.shape[-1]

    print("Creating the different pipelines")
    clf = basearchi(windows_size = n_samples_windows, n_channel_input = X_train.shape[2])
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    batchsize = 64 #128 # 64 for burst
    epochs = 20 #45 # 20 for burst

    print("Fitting")
    start = time.time()
    lr = 1e-3
    weight_decay = 1e-4
    optimizer = keras.optimizers.Adam(learning_rate=lr, amsgrad=True)
    clf.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
    history_list.append(clf.fit(np.array(x_train), y_train,
                    batch_size=batchsize, epochs=epochs,
                    validation_data=(np.array(x_val), y_val), shuffle=True))
    CNN_tps_train_code_perso[i] = time.time() - start

    print("getting accuracy of participant ", i)
    start = time.time()
    y_pred = clf.predict(X_test)[:,0]
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y[0] == 0 else 1 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    CNN_accuracy_perso[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    labels_pred_accumul, _, mean_long_accumul = mpaa(
        y_pred_norm, codes, min_len=30, fps=60, consecutive=50, window_size=window_size
    )
    print("meannnnnn long",np.mean(mean_long_accumul))
    CNN_tps_test_code_perso[i] = time.time() - start
    CNN_accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(CNN_accuracy_perso)
print(CNN_tps_train_code_perso)
print(CNN_tps_test_code_perso)
print(CNN_accuracy_code_perso)
# pd.DataFrame(CNN_accuracy_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/score/DA_score.csv")
# pd.DataFrame(CNN_accuracy_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/score_code/DA_score_code.csv")
# pd.DataFrame(CNN_tps_train_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/temps_train_code/DA_tps_train_code.csv")
# pd.DataFrame(CNN_tps_test_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/temps_test_code/DA_tps_test_code.csv")



TL to the participant :  0
(198930, 1, 8, 150)
(198930,)
(6270, 1, 8, 150)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
256/256 [==============================] - 3s 10ms/step - loss: 0.6607 - accuracy: 0.6143 - val_loss: 0.5705 - val_accuracy: 0.7036
Epoch 2/20
256/256 [==============================] - 3s 10ms/step - loss: 0.5938 - accuracy: 0.6953 - val_loss: 0.5520 - val_accuracy: 0.7240
Epoch 3/20
256/256 [==============================] - 2s 9ms/step - loss: 0.5763 - accuracy: 0.7079 - val_loss: 0.5348 - val_accuracy: 0.7383
Epoch 4/20
256/256 [==============================] - 2s 10ms/step - loss: 0.5705 - accuracy: 0.7160 - val_loss: 0.5336 - val_accuracy: 0.7317
Epoch 5/20
256/256 [==============================] - 2s 10ms/step - loss: 0.5698 - accuracy: 0.7172 - val_loss: 0.5343 - val_accuracy: 0.7388
Epoch 6/20
256/256 [==============================] - 2s 10ms/step - loss: 0.5643 - accuracy: 0.7201 - val_loss: 0.5362 - val_accur

In [9]:
print("with 0.3s",np.mean([0.89, 0.91, 0.31, 0.89, 0.93, 0.93, 0.87, 1,   0.78, 0.84, 0.84, 0.71, 0.85, 0.95,
 0.53, 0.96, 0.87, 0.96, 0.16, 0.87, 0.8,  0.38, 0.51, 0.95]))
print("with 0.25s",np.mean([0.78, 0.91, 0.29, 0.91, 0.78, 0.82, 0.91, 0.95, 0.82, 0.8,  0.8,  0.85, 0.73, 0.87,
 0.55, 0.93, 0.85, 0.93, 0.16, 0.71, 0.82, 0.45, 0.36, 0.93]))

with 0.3s 0.77875
with 0.25s 0.74625


## XDAWn + TS + LDA

### XTL SS

In [53]:
n_cal = 7
n_class = 5
nb_part = len(participants)
CNN_accuracy_code_perso = np.zeros(nb_part)
CNN_tps_train_code_perso = np.zeros(nb_part)
CNN_tps_test_code_perso = np.zeros(nb_part)
CNN_accuracy_perso = np.zeros(nb_part)
nb_samples_windows = int((2.2-window_size)*n_class*n_cal*sfreq)
history_list = []


for i in range(nb_part):
    print("TL to the participant : ", i)

    X_train = X[i][:nb_samples_windows]
    Y_train = y[i][:nb_samples_windows]
    # Y_train = np.vstack((Y_train,np.abs(1-Y_train))).T
    domains_train = domains[i][:nb_samples_windows]
    X_test = X[i][nb_samples_windows:]
    Y_test = y[i][nb_samples_windows:]
    labels_code_test = labels_code_list[i][n_cal*n_class:]

    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    rus = RandomUnderSampler()
    counter=np.array(range(0,len(Y_train))).reshape(-1,1)
    index,_ = rus.fit_resample(counter,Y_train[:])
    X_train = np.squeeze(X_train[index,:,:], axis=1)
    Y_train = np.squeeze(Y_train[index])
    n_samples_windows = X_train.shape[-1]
    print(X_train.shape)
    print(X_test.shape)

    print("Creating the different pipelines")
    clf = make_pipeline(XdawnCovariances(nfilter=8, estimator="lwf", xdawn_estimator="lwf",classes=[1]),
                        TangentSpace(), LDA(solver="lsqr", shrinkage="auto"))
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42, shuffle=True)
    batchsize = 64 #128 # 64 for burst
    epochs = 20 #45 # 20 for burst

    print("Fitting")
    start = time.time()
    lr = 1e-3
    weight_decay = 1e-4
    # optimizer = keras.optimizers.Adam(learning_rate=lr, amsgrad=True)
    # clf.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
    history_list.append(clf.fit(np.array(x_train), y_train))#,
                    # batch_size=batchsize, epochs=epochs,
                    # validation_data=(np.array(x_val), y_val), shuffle=True))
    CNN_tps_train_code_perso[i] = time.time() - start

    print("getting accuracy of participant ", i)
    start = time.time()
    y_pred = clf.predict(X_test)
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y == 0 else 1 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    CNN_accuracy_perso[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    labels_pred_accumul, _, mean_long_accumul = mpaa(
        y_pred_norm, codes, min_len=30, sfreq=sfreq, consecutive=40, window_size=window_size
    )
    print("meannnnnn long",np.mean(mean_long_accumul))
    CNN_tps_test_code_perso[i] = time.time() - start
    CNN_accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(CNN_accuracy_perso)
print(CNN_tps_train_code_perso)
print(CNN_tps_test_code_perso)
print(CNN_accuracy_code_perso)
print(np.mean(CNN_accuracy_code_perso))
# pd.DataFrame(CNN_accuracy_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/score/SS_score.csv")
# pd.DataFrame(CNN_accuracy_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/score_code/SS_score_code.csv")
# pd.DataFrame(CNN_tps_train_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/temps_train_code/SS_tps_train_code.csv")
# pd.DataFrame(CNN_tps_test_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/temps_test_code/SS_tps_test_code.csv")


TL to the participant :  0
balancing the number of ones and zeros
(1456, 8, 60)
(18720, 8, 60)
Creating the different pipelines
Fitting
getting accuracy of participant  0
meannnnnn long 1.3257575757575757

TL to the participant :  1
balancing the number of ones and zeros
(1456, 8, 60)
(18720, 8, 60)
Creating the different pipelines
Fitting
getting accuracy of participant  1
meannnnnn long 1.3326666666666669
TL to the participant :  2
balancing the number of ones and zeros
(1456, 8, 60)
(18720, 8, 60)
Creating the different pipelines
Fitting
getting accuracy of participant  2


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.457638888888889
TL to the participant :  3
balancing the number of ones and zeros
(1456, 8, 60)
(18720, 8, 60)
Creating the different pipelines
Fitting
getting accuracy of participant  3
meannnnnn long 1.344949494949495
TL to the participant :  4
balancing the number of ones and zeros
(1456, 8, 60)
(18720, 8, 60)
Creating the different pipelines
Fitting
getting accuracy of participant  4


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.392361111111111
TL to the participant :  5
balancing the number of ones and zeros
(1456, 8, 60)
(18720, 8, 60)
Creating the different pipelines
Fitting
getting accuracy of participant  5


...

meannnnnn long 1.3878205128205126
TL to the participant :  16
balancing the number of ones and zeros
(1456, 8, 60)
(18720, 8, 60)
Creating the different pipelines
Fitting
getting accuracy of participant  16
meannnnnn long 1.3212121212121213
TL to the participant :  17
balancing the number of ones and zeros
(1456, 8, 60)
(18720, 8, 60)
Creating the different pipelines
Fitting
getting accuracy of participant  17


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.355952380952381
TL to the participant :  18
balancing the number of ones and zeros
(1456, 8, 60)
(18720, 8, 60)
Creating the different pipelines
Fitting
getting accuracy of participant  18
meannnnnn long 1.33763440860215
TL to the participant :  19
balancing the number of ones and zeros
(1456, 8, 60)
(18720, 8, 60)
Creating the different pipelines
Fitting
getting accuracy of participant  19


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.4513333333333334
TL to the participant :  20
balancing the number of ones and zeros
(1456, 8, 60)
(18720, 8, 60)
Creating the different pipelines
Fitting
getting accuracy of participant  20
meannnnnn long 1.4294117647058824
TL to the participant :  21
balancing the number of ones and zeros
(1456, 8, 60)
(18720, 8, 60)
Creating the different pipelines
Fitting
getting accuracy of participant  21
meannnnnn long 1.427450980392157
TL to the participant :  22
balancing the number of ones and zeros
(1456, 8, 60)
(18720, 8, 60)
Creating the different pipelines
Fitting
getting accuracy of participant  22
meannnnnn long 1.34469696969697
TL to the participant :  23
balancing the number of ones and zeros
(1456, 8, 60)
(18720, 8, 60)
Creating the different pipelines
Fitting
getting accuracy of participant  23
meannnnnn long 1.3554054054054052
[0.72928779 0.66954103 0.62741782 0.70062332 0.68527784 0.72074855
 0.68826867 0.7429981  0.5908011  0.54180177 0.6348949  0.69451588
 0.6241

In [66]:
print("with 0.3s",np.mean([1, 0.98, 0.85, 0.92, 0.98, 0.95, 0.82, 0.98, 0.9,  0.2,  0.9,  0.88, 0.8,  0.95,
 0.78, 0.92, 0.98, 0.95, 0.7,  0.88, 1,   0.88, 0.68, 1  ]))
print("with 0.25s",np.mean([0.92 ,0.92, 0.85, 0.92, 0.98, 0.88, 0.65, 0.98, 0.75, 0.22, 0.85, 0.92, 0.78, 0.88,
 0.57, 1,   0.98, 0.9,  0.78, 0.75, 1,   0.98 ,0.65, 1  ]))

with 0.3s 0.8700000000000001
with 0.25s 0.8379166666666666


### XTL DG

In [25]:

n_cal = 4
n_class = 5
nb_part = len(participants)
CNN_accuracy_code_perso = np.zeros(nb_part)
CNN_tps_train_code_perso = np.zeros(nb_part)
CNN_tps_test_code_perso = np.zeros(nb_part)
CNN_accuracy_perso = np.zeros(nb_part)
nb_samples_windows = int((2.2-window_size)*n_class*n_cal*60)
history_list = []


for i in range(nb_part):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(nb_part) if j!=i]

    X_train = np.concatenate(X[ind2take]).reshape(-1,X.shape[-2],X.shape[-1])
    Y_train = np.concatenate(y[ind2take]).reshape(-1)
    domains_train = np.concatenate(domains[ind2take]).reshape(-1)
    X_test = X[i]
    Y_test = y[i]
    labels_code_test = labels_code_list[i]

    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train,Y_train,domains_train = balance(X_train,Y_train,domains_train)
    n_samples_windows = X_train.shape[-1]

    print("Creating the different pipelines")
    clf = make_pipeline(XdawnCovariances(nfilter=16, estimator="lwf", xdawn_estimator="lwf",classes=[1]),
                        TangentSpace(), LDA(solver="lsqr", shrinkage="auto"))
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    batchsize = 64 #128 # 64 for burst
    epochs = 20 #45 # 20 for burst

    print("Fitting")
    start = time.time()
    lr = 1e-3
    weight_decay = 1e-4
    optimizer = keras.optimizers.Adam(learning_rate=lr, amsgrad=True)
    # clf.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
    history_list.append(clf.fit(np.array(x_train), y_train))#,
                    # batch_size=batchsize, epochs=epochs,
                    # validation_data=(np.array(x_val), y_val), shuffle=True))
    CNN_tps_train_code_perso[i] = time.time() - start

    print("getting accuracy of participant ", i)
    start = time.time()
    y_pred = clf.predict(X_test)
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y == 0 else 1 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    CNN_accuracy_perso[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    labels_pred_accumul, _, mean_long_accumul = mpaa(
        y_pred_norm, codes, min_len=30, fps=60, consecutive=50, window_size=window_size
    )
    print("meannnnnn long",np.mean(mean_long_accumul))
    CNN_tps_test_code_perso[i] = time.time() - start
    CNN_accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(CNN_accuracy_perso)
print(CNN_tps_train_code_perso)
print(CNN_tps_test_code_perso)
print(CNN_accuracy_code_perso)
# pd.DataFrame(CNN_accuracy_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/score/DG_score.csv")
# pd.DataFrame(CNN_accuracy_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/score_code/DG_score_code.csv")
# pd.DataFrame(CNN_tps_train_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/temps_train_code/DG_tps_train_code.csv")
# pd.DataFrame(CNN_tps_test_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/temps_test_code/DG_tps_test_code.csv")



TL to the participant :  0
(201825, 8, 125)
(201825,)
(8775, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  0
meannnnnn long 1.4072916666666666
TL to the participant :  1
(201825, 8, 125)
(201825,)
(8775, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  1
meannnnnn long 1.4546099290780141
TL to the participant :  2
(201825, 8, 125)
(201825,)
(8775, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  2
meannnnnn long 1.4912037037037038
TL to the participant :  3
(201825, 8, 125)
(201825,)
(8775, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  3
meannnnnn long 1.4608024691358024
TL to the participant :  4
(201825, 8, 125)
(201825,)
(8775, 8, 125)
balancing the number of ones and zeros
Creating the differe

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.4939393939393937
TL to the participant :  10
(201825, 8, 125)
(201825,)
(8775, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  10
meannnnnn long 1.388095238095238
TL to the participant :  11
(201825, 8, 125)
(201825,)
(8775, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  11
meannnnnn long 1.4722222222222225
TL to the participant :  12
(201825, 8, 125)
(201825,)
(8775, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  12
meannnnnn long 1.4231481481481483
TL to the participant :  13
(201825, 8, 125)
(201825,)
(8775, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  13
meannnnnn long 1.4160377358490563
TL to the participant :  14
(201825, 8, 125)
(201825,)
(8775, 8, 125)
balancing the num

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.4308176100628927
TL to the participant :  21
(201825, 8, 125)
(201825,)
(8775, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  21
meannnnnn long 1.4462499999999998
TL to the participant :  22
(201825, 8, 125)
(201825,)
(8775, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  22
meannnnnn long 1.4557471264367818
TL to the participant :  23
(201825, 8, 125)
(201825,)
(8775, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  23
meannnnnn long 1.3948717948717948
[0.74740608 0.76973763 0.55456172 0.78729875 0.74519976 0.78008348
 0.74230769 0.82242099 0.71362552 0.64648181 0.68950507 0.65104353
 0.68416816 0.75986881 0.60387597 0.81487776 0.72593918 0.79007156
 0.48273703 0.71815742 0.70721527 0.66138939 0.57042338 0.82429934]
[45.87342191 45.59355545 45.70195389 

In [67]:
print("with 0.3s",np.mean([0.81, 0.88, 0.29, 0.97, 0.91, 0.88, 0.83, 0.97, 0.89, 0.71, 0.8, 0.67, 0.83, 0.93,
 0.61, 0.93, 0.83, 0.89, 0.15, 0.88, 0.75, 0.6,  0.51, 0.92]))
print("with 0.25s",np.mean([0.87, 0.91 ,0.37, 0.92, 0.83, 0.77, 0.8,  0.96, 0.84, 0.63, 0.65, 0.65, 0.72, 0.96,
 0.48, 0.93, 0.84 ,0.93 ,0.16, 0.85, 0.75, 0.52, 0.41, 0.92]))

with 0.3s 0.7683333333333332
with 0.25s 0.7362500000000001


### XTL DA

In [29]:

n_cal = 4
n_class = 5
nb_part = len(participants)
CNN_accuracy_code_perso = np.zeros(nb_part)
CNN_tps_train_code_perso = np.zeros(nb_part)
CNN_tps_test_code_perso = np.zeros(nb_part)
CNN_accuracy_perso = np.zeros(nb_part)
nb_samples_windows = int((2.2-window_size)*n_class*n_cal*60)
history_list = []


for i in range(nb_part):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(nb_part) if j!=i]

    X_train = np.concatenate([X[ind2take].reshape(-1,X.shape[-2],X.shape[-1]),X[i][:nb_samples_windows]]).reshape(-1,X.shape[-2],X.shape[-1])
    Y_train = np.concatenate([y[ind2take].reshape(-1),y[i][:nb_samples_windows]]).reshape(-1)
    domains_train = np.concatenate([domains[ind2take].reshape(-1),domains[i][:nb_samples_windows]]).reshape(-1)
    X_test = X[i][nb_samples_windows:]
    Y_test = y[i][nb_samples_windows:]
    labels_code_test = labels_code_list[i][n_cal*n_class:]

    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)
    X_std = X_train.std(axis=0)
    X_train /= X_std + 1e-8
    X_std = X_test.std(axis=0)
    X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train,Y_train,domains_train = balance(X_train,Y_train,domains_train)
    n_samples_windows = X_train.shape[-1]

    print("Creating the different pipelines")
    clf = make_pipeline(XdawnCovariances(nfilter=8, estimator="lwf", xdawn_estimator="lwf",classes=[1]),
                        TangentSpace(), LDA(solver="lsqr", shrinkage="auto"))
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    batchsize = 64 #128 # 64 for burst
    epochs = 20 #45 # 20 for burst

    print("Fitting")
    start = time.time()
    lr = 1e-3
    weight_decay = 1e-4
    # optimizer = keras.optimizers.Adam(learning_rate=lr, amsgrad=True)
    # clf.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
    history_list.append(clf.fit(np.array(x_train), y_train))#,
                    # batch_size=batchsize, epochs=epochs,
                    # validation_data=(np.array(x_val), y_val), shuffle=True))
    CNN_tps_train_code_perso[i] = time.time() - start

    print("getting accuracy of participant ", i)
    start = time.time()
    y_pred = clf.predict(X_test)
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([0 if y == 0 else 1 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    CNN_accuracy_perso[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    labels_pred_accumul, _, mean_long_accumul = mpaa(
        y_pred_norm, codes, min_len=30, fps=60, consecutive=50, window_size=window_size
    )
    print("meannnnnn long",np.mean(mean_long_accumul))
    CNN_tps_test_code_perso[i] = time.time() - start
    CNN_accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(CNN_accuracy_perso)
print(CNN_tps_train_code_perso)
print(CNN_tps_test_code_perso)
print(CNN_accuracy_code_perso)
# pd.DataFrame(CNN_accuracy_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/score/DA_score.csv")
# pd.DataFrame(CNN_accuracy_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/score_code/DA_score_code.csv")
# pd.DataFrame(CNN_tps_train_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/temps_train_code/DA_tps_train_code.csv")
# pd.DataFrame(CNN_tps_test_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/CNN/temps_test_code/DA_tps_test_code.csv")



TL to the participant :  0
(204165, 8, 125)
(204165,)
(6435, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  0
meannnnnn long 1.4237499999999998
TL to the participant :  1
(204165, 8, 125)
(204165,)
(6435, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  1
meannnnnn long 1.4864035087719298
TL to the participant :  2
(204165, 8, 125)
(204165,)
(6435, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  2
meannnnnn long 1.4613333333333332
TL to the participant :  3
(204165, 8, 125)
(204165,)
(6435, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  3
meannnnnn long 1.47280701754386
TL to the participant :  4
(204165, 8, 125)
(204165,)
(6435, 8, 125)
balancing the number of ones and zeros
Creating the different

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.4102272727272729
TL to the participant :  18
(204165, 8, 125)
(204165,)
(6435, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  18
meannnnnn long 1.5224637681159423
TL to the participant :  19
(204165, 8, 125)
(204165,)
(6435, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  19
meannnnnn long 1.4333333333333331
TL to the participant :  20
(204165, 8, 125)
(204165,)
(6435, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  20


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.406910569105691
TL to the participant :  21
(204165, 8, 125)
(204165,)
(6435, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  21
meannnnnn long 1.4014492753623187
TL to the participant :  22
(204165, 8, 125)
(204165,)
(6435, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  22
meannnnnn long 1.4717948717948721
TL to the participant :  23
(204165, 8, 125)
(204165,)
(6435, 8, 125)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  23
meannnnnn long 1.3967391304347827
[0.75390307 0.77845991 0.56285575 0.7856562  0.74247845 0.79081965
 0.73227354 0.81541714 0.7123516  0.66014799 0.70434217 0.66299398
 0.67193853 0.76638478 0.62554887 0.81651488 0.74398276 0.79130753
 0.50215482 0.72528053 0.75764352 0.66181493 0.57163766 0.83180192]
[31.848665   31.73069596 33.48490167 3

In [30]:
np.mean(CNN_accuracy_code_perso)

0.7504166666666666

In [3]:
print("with 0.3s",np.mean([0.85, 0.85, 0.24, 0.96, 0.95, 0.95, 0.87, 0.91, 0.87, 0.8, 0.85, 0.69 ,0.84, 0.96,
 0.65, 0.96, 0.91, 0.96 ,0.11 ,0.87 ,0.8,  0.6,  0.65, 0.95]))
print("with 0.25s",np.mean([0.84, 0.85, 0.35, 0.93 ,0.91 ,0.8,  0.82, 0.95, 0.82, 0.73 ,0.73 ,0.73, 0.58, 0.95,
 0.51, 0.93, 0.87, 0.89, 0.27, 0.76, 0.85, 0.58, 0.45, 0.98]))

with 0.3s 0.7937500000000001
with 0.25s 0.7533333333333333


## SPD test

In [88]:
X_riem = np.zeros((X.shape[0],X.shape[1],16,16))
y_riem = np.zeros((X.shape[0],X.shape[1]))
for ind_i,i in enumerate(participants):
    events, event_id = mne.events_from_annotations(raw_eeglab[ind_i])
    print(event_id)
    epochs = mne.Epochs(raw_eeglab[ind_i],events,event_id,tmin=0.0,tmax=window_size,baseline=(0,0))
    temp_X = epochs.get_data()[:,:,:-1]
    
    y_riem[ind_i] = epochs.events[...,-1]-1
    # print(temp_X.shape)
    # X_riem[ind_i] = compute_riemannian_alignment(Euc2SPD(X[ind_i],y[ind_i]), mean=None, dtype='covmat')
    X_riem[ind_i] = compute_riemannian_alignment(Euc2SPD(temp_X,y_riem[ind_i]), mean=None, dtype='covmat')
    print(X_riem[ind_i].shape)

Used Annotations descriptions: ['0', '1']
{'0': 1, '1': 2}
Not setting metadata
8775 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 8775 events and 126 original time points ...
0 bad epochs dropped
(8775, 16, 16)
Used Annotations descriptions: ['0', '1']
{'0': 1, '1': 2}
Not setting metadata
8775 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 8775 events and 126 original time points ...
0 bad epochs dropped
(8775, 16, 16)
Used Annotations descriptions: ['0', '1']
{'0': 1, '1': 2}
Not setting metadata
8775 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 8775 events and 126 original time points ...
0 bad epochs dropped
(8775, 16, 16)
Used Annotations descriptions: ['0', '1']
{'0': 1, '1': 2}
Not setting metadata
8775 matching events found
Applying basel

In [59]:
X_riem.shape

(24, 73125, 16, 16)

### SPD SS

In [89]:
n_cal = 7
n_class = 5
nb_part = len(participants)
SPD_accuracy_code_perso = np.zeros(nb_part)
SPD_tps_train_code_perso = np.zeros(nb_part)
SPD_tps_test_code_perso = np.zeros(nb_part)
SPD_accuracy_perso = np.zeros(nb_part)
nb_samples_windows = int((2.2-window_size)*n_class*n_cal*60)
history_list = []


for i in range(nb_part):
    print("TL to the participant : ", i)

    X_train = X_riem[i][:nb_samples_windows]
    Y_train = y_riem[i][:nb_samples_windows]
    # Y_train = np.vstack((Y_train,np.abs(1-Y_train))).T
    domains_train = domains[i][:nb_samples_windows]
    X_test = X_riem[i][nb_samples_windows:]
    Y_test = y_riem[i][nb_samples_windows:]
    labels_code_test = labels_code_list[i][n_cal*n_class:]

    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)
    # X_std = X_train.std(axis=0)
    # X_train /= X_std + 1e-8
    # X_std = X_test.std(axis=0)
    # X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train,Y_train,domains_train = balance(X_train,Y_train,domains_train)

    print("Creating the different pipelines")
    clf = SPDNet_Tensorflow(bimap_dims=[16,8,4])
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    batchsize = 64 #128 # 64 for burst
    epochs = 20 #45 # 20 for burst

    print("Fitting")
    start = time.time()
    lr = 1e-3
    weight_decay = 1e-4
    history_list.append(clf.fit(np.array(x_train), y_train,
                    batch_size=batchsize, epochs=epochs,
                    validation_data=(np.array(x_val), y_val), shuffle=True,verbose=False))
    SPD_tps_train_code_perso[i] = time.time() - start

    print("getting accuracy of participant ", i)
    start = time.time()
    y_pred = clf.predict(X_test)
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([1 if (y >= 0.5) else 0 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    SPD_accuracy_perso[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    labels_pred_accumul, _, mean_long_accumul = mpaa(
        y_pred_norm, codes, min_len=30, sfreq=60, consecutive=50, window_size=window_size
    )
    print("meannnnnn long",np.mean(mean_long_accumul))
    SPD_tps_test_code_perso[i] = time.time() - start
    SPD_accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(SPD_accuracy_perso)
print(SPD_tps_train_code_perso)
print(SPD_tps_test_code_perso)
print(SPD_accuracy_code_perso)
print(np.mean(SPD_accuracy_code_perso))
# pd.DataFrame(SPD_accuracy_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPD/score/DG_score.csv")
# pd.DataFrame(SPD_accuracy_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPD/score_code/DG_score_code.csv")
# pd.DataFrame(SPD_tps_train_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPD/temps_train_code/DG_tps_train_code.csv")
# pd.DataFrame(SPD_tps_test_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPD/temps_test_code/DG_tps_test_code.csv")


TL to the participant :  0
(4095, 16, 16)
(4095,)
(4680, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  0
147/147 [==============================] - 0s 3ms/step
meannnnnn long 1.4197916666666666
TL to the participant :  1
(4095, 16, 16)
(4095,)
(4680, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  1
147/147 [==============================] - 1s 3ms/step


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.3933333333333335
TL to the participant :  2
(4095, 16, 16)
(4095,)
(4680, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  2
147/147 [==============================] - 1s 3ms/step


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.413793103448276
TL to the participant :  3
(4095, 16, 16)
(4095,)
(4680, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  3
147/147 [==============================] - 1s 3ms/step


...

meannnnnn long 1.5142857142857142
TL to the participant :  18
(4095, 16, 16)
(4095,)
(4680, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  18
147/147 [==============================] - 1s 3ms/step
meannnnnn long 1.4958333333333336
TL to the participant :  19
(4095, 16, 16)
(4095,)
(4680, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  19
147/147 [==============================] - 1s 3ms/step


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.508888888888889
TL to the participant :  20
(4095, 16, 16)
(4095,)
(4680, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  20
147/147 [==============================] - 1s 3ms/step


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.4063063063063062
TL to the participant :  21
(4095, 16, 16)
(4095,)
(4680, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  21
147/147 [==============================] - 1s 3ms/step
meannnnnn long 1.482051282051282
TL to the participant :  22
(4095, 16, 16)
(4095,)
(4680, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  22
147/147 [==============================] - 1s 3ms/step
meannnnnn long 1.4396825396825395
TL to the participant :  23
(4095, 16, 16)
(4095,)
(4680, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
getting accuracy of participant  23
147/147 [==============================] - 1s 3ms/step
meannnnnn long 1.3905405405405409
[0.81853757 0.72260733 0.78052326 0.76453488 0.72299866 0.6644678
 0.75067084 0.77454159 0.6168381  0.64859123 0.77224955 0.65373435
 0.67285331 0.7218805

In [ ]:
print("with 0.3s and filtre on 0 et 1",np.mean([1, 0.88, 0.85, 0.88, 0.95, 0.95, 0.68, 0.92, 0.9,  0.68, 0.92, 0.75, 0.85, 0.98,
 0.78, 0.85, 0.98, 0.92, 0.78, 0.78, 1, 0.6,  0.75, 1 ]))
print("with timepoint wise 500Hz",np.mean([1,1,0.98,0.98,0.98,1,0.85,1,0.95,0.85,0.95,0.92,0.98,1.
,0.8,1,1,0.92,0.88,0.95,1,0.98,0.82,1]))
print("with 0.25s and filter on 0 et 1",np.mean([1, 0.92, 0.88, 0.9,  0.98, 1,   0.88, 0.95, 0.92, 0.82, 0.9,  0.85, 0.92, 0.95,
 0.8,  0.98, 0.98, 0.98, 0.85, 0.8,  0.98, 0.95, 0.75, 1  ]))

with 0.3s 0.8595833333333335
with 0.25s 0.9141666666666666


### SPD DG

In [90]:
n_cal = 4
n_class = 5
nb_part = len(participants)
SPD_accuracy_code_perso = np.zeros(nb_part)
SPD_tps_train_code_perso = np.zeros(nb_part)
SPD_tps_test_code_perso = np.zeros(nb_part)
SPD_accuracy_perso = np.zeros(nb_part)
nb_samples_windows = int((2.2-window_size)*n_class*n_cal*60)
history_list = []


for i in range(nb_part):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(nb_part) if j!=i]

    X_train = np.concatenate(X_riem[ind2take]).reshape(-1,X_riem.shape[-2],X_riem.shape[-1])
    Y_train = np.concatenate(y_riem[ind2take]).reshape(-1)
    domains_train = np.concatenate(domains[ind2take]).reshape(-1)
    X_test = X_riem[i]
    Y_test = y_riem[i]
    labels_code_test = labels_code_list[i]

    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)
    # X_std = X_train.std(axis=0)
    # X_train /= X_std + 1e-8
    # X_std = X_test.std(axis=0)
    # X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train,Y_train,domains_train = balance(X_train,Y_train,domains_train)

    print("Creating the different pipelines")
    clf = SPDNet_Tensorflow(bimap_dims=[16,8,4])
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    batchsize = 64 #128 # 64 for burst
    epochs = 20 #45 # 20 for burst

    print("Fitting")
    start = time.time()
    lr = 1e-3
    weight_decay = 1e-4
    history_list.append(clf.fit(np.array(x_train), y_train,
                    batch_size=batchsize, epochs=epochs,
                    validation_data=(np.array(x_val), y_val), shuffle=True))
    SPD_tps_train_code_perso[i] = time.time() - start

    print("getting accuracy of participant ", i)
    start = time.time()
    y_pred = clf.predict(X_test)
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([1 if (y >= 0.5) else 0 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    SPD_accuracy_perso[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    labels_pred_accumul, _, mean_long_accumul = mpaa(
        y_pred_norm, codes, min_len=30, sfreq=60, consecutive=50, window_size=window_size
    )
    print("meannnnnn long",np.mean(mean_long_accumul))
    SPD_tps_test_code_perso[i] = time.time() - start
    SPD_accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(SPD_accuracy_perso)
print(SPD_tps_train_code_perso)
print(SPD_tps_test_code_perso)
print(SPD_accuracy_code_perso)
# pd.DataFrame(SPD_accuracy_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPD/score/DG_score.csv")
# pd.DataFrame(SPD_accuracy_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPD/score_code/DG_score_code.csv")
# pd.DataFrame(SPD_tps_train_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPD/temps_train_code/DG_tps_train_code.csv")
# pd.DataFrame(SPD_tps_test_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPD/temps_test_code/DG_tps_test_code.csv")


TL to the participant :  0
(201825, 16, 16)
(201825,)
(8775, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
253/253 [==============================] - 4s 13ms/step - loss: 0.5955 - sparse_categorical_accuracy: 0.6891 - val_loss: 0.4988 - val_sparse_categorical_accuracy: 0.7781
Epoch 2/20
253/253 [==============================] - 3s 13ms/step - loss: 0.4785 - sparse_categorical_accuracy: 0.7849 - val_loss: 0.4553 - val_sparse_categorical_accuracy: 0.7804
Epoch 3/20
253/253 [==============================] - 3s 13ms/step - loss: 0.4602 - sparse_categorical_accuracy: 0.7915 - val_loss: 0.4447 - val_sparse_categorical_accuracy: 0.7993
Epoch 4/20
253/253 [==============================] - 3s 13ms/step - loss: 0.4541 - sparse_categorical_accuracy: 0.7903 - val_loss: 0.4410 - val_sparse_categorical_accuracy: 0.7993
Epoch 5/20
253/253 [==============================] - 3s 13ms/step - loss: 0.4522 - sparse_categorical_accuracy: 0.7935 - val_l

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.3564444444444446
TL to the participant :  1
(201825, 16, 16)
(201825,)
(8775, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
253/253 [==============================] - 4s 14ms/step - loss: 0.6088 - sparse_categorical_accuracy: 0.6810 - val_loss: 0.5291 - val_sparse_categorical_accuracy: 0.7687
Epoch 2/20
253/253 [==============================] - 4s 14ms/step - loss: 0.4931 - sparse_categorical_accuracy: 0.7775 - val_loss: 0.4743 - val_sparse_categorical_accuracy: 0.7926
Epoch 3/20
253/253 [==============================] - 4s 14ms/step - loss: 0.4682 - sparse_categorical_accuracy: 0.7902 - val_loss: 0.4764 - val_sparse_categorical_accuracy: 0.7781
Epoch 4/20
253/253 [==============================] - 4s 14ms/step - loss: 0.4629 - sparse_categorical_accuracy: 0.7937 - val_loss: 0.4739 - val_sparse_categorical_accuracy: 0.7832
Epoch 5/20
253/253 [==============================] - 4s 14ms/step - loss: 0.4596 - sparse_ca

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.4787878787878788
TL to the participant :  2
(201825, 16, 16)
(201825,)
(8775, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
253/253 [==============================] - 4s 15ms/step - loss: 0.5449 - sparse_categorical_accuracy: 0.7352 - val_loss: 0.4893 - val_sparse_categorical_accuracy: 0.7837
Epoch 2/20
253/253 [==============================] - 4s 15ms/step - loss: 0.4688 - sparse_categorical_accuracy: 0.7877 - val_loss: 0.4864 - val_sparse_categorical_accuracy: 0.7815
Epoch 3/20
253/253 [==============================] - 4s 14ms/step - loss: 0.4591 - sparse_categorical_accuracy: 0.7905 - val_loss: 0.4769 - val_sparse_categorical_accuracy: 0.7893
Epoch 4/20
253/253 [==============================] - 4s 15ms/step - loss: 0.4541 - sparse_categorical_accuracy: 0.7951 - val_loss: 0.4729 - val_sparse_categorical_accuracy: 0.7915
Epoch 5/20
253/253 [==============================] - 4s 15ms/step - loss: 0.4486 - sparse_ca

...

meannnnnn long 1.3628378378378379
TL to the participant :  21
(201825, 16, 16)
(201825,)
(8775, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
253/253 [==============================] - 4s 15ms/step - loss: 0.5282 - sparse_categorical_accuracy: 0.7441 - val_loss: 0.4807 - val_sparse_categorical_accuracy: 0.7776
Epoch 2/20
253/253 [==============================] - 4s 14ms/step - loss: 0.4595 - sparse_categorical_accuracy: 0.7923 - val_loss: 0.4664 - val_sparse_categorical_accuracy: 0.7809
Epoch 3/20
253/253 [==============================] - 4s 15ms/step - loss: 0.4467 - sparse_categorical_accuracy: 0.7972 - val_loss: 0.4526 - val_sparse_categorical_accuracy: 0.7876
Epoch 4/20
253/253 [==============================] - 4s 14ms/step - loss: 0.4418 - sparse_categorical_accuracy: 0.8026 - val_loss: 0.4457 - val_sparse_categorical_accuracy: 0.7988
Epoch 5/20
253/253 [==============================] - 4s 15ms/step - loss: 0.4395 - sparse_c

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.4374149659863944
TL to the participant :  22
(201825, 16, 16)
(201825,)
(8775, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
253/253 [==============================] - 4s 14ms/step - loss: 0.5601 - sparse_categorical_accuracy: 0.7273 - val_loss: 0.5027 - val_sparse_categorical_accuracy: 0.7648
Epoch 2/20
253/253 [==============================] - 4s 14ms/step - loss: 0.4784 - sparse_categorical_accuracy: 0.7779 - val_loss: 0.4842 - val_sparse_categorical_accuracy: 0.7676
Epoch 3/20
253/253 [==============================] - 4s 14ms/step - loss: 0.4665 - sparse_categorical_accuracy: 0.7881 - val_loss: 0.4829 - val_sparse_categorical_accuracy: 0.7787
Epoch 4/20
253/253 [==============================] - 4s 14ms/step - loss: 0.4601 - sparse_categorical_accuracy: 0.7931 - val_loss: 0.4692 - val_sparse_categorical_accuracy: 0.7876
Epoch 5/20
253/253 [==============================] - 4s 14ms/step - loss: 0.4562 - sparse_c

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.3655092592592595
[0.91216458 0.67438879 0.73962433 0.84159213 0.81753131 0.83205128
 0.80578414 0.88363148 0.70730471 0.69293381 0.82104949 0.77122838
 0.76007752 0.8245975  0.70026834 0.83008348 0.88419797 0.72802624
 0.77602862 0.73315444 0.90286225 0.77400119 0.69060823 0.90745379]
[68.90085959 74.82860589 71.10506964 66.93248701 68.04010344 69.07430387
 70.47576642 66.59381413 70.49327922 70.82116604 70.07322288 73.27170658
 71.62636089 71.58326411 75.94529629 78.78629732 80.35693336 83.98412776
 77.79184031 76.91834998 75.9222157  74.54997492 73.76206207 74.37730289]
[1.8823154  2.24804831 3.17404246 3.04600167 2.98578763 3.1433022
 3.15248537 3.47751951 3.39072967 3.42717719 2.00916481 3.28504443
 3.32829642 3.11451459 2.33289337 2.15232849 2.08214951 2.3092761
 2.27099681 2.31841135 2.04596019 2.17342353 2.24336386 1.97573876]
[1.   0.87 0.89 0.99 0.99 0.99 0.93 0.96 0.81 0.79 0.99 0.93 0.92 0.99
 0.73 0.89 0.95 0.8  0.81 0.84 1.   0.91 0.77 0.99]


In [92]:
print("with 0.3s",np.mean([1, 0.96, 0.88, 1,   1,   0.99, 0.92, 0.99, 0.84, 0.27, 0.97, 0.89, 0.95, 0.99,
 0.73, 0.95, 0.97, 0.89 ,0.85, 0.91, 1, 0.93, 0.8 , 0.97]))
print("with timepoint wise 500HZ",np.mean([0.99,0.92,0.93,0.99,0.99,0.99,0.95,0.97,0.89,0.79,0.97,0.93,0.96,0.97
,0.81,0.97,0.97,0.81,0.8,0.89,1.,  0.97,0.85,0.99]))
np.mean([1, 0.93, 0.89 ,0.99, 0.96, 0.96, 0.91 ,0.95 ,0.89 ,0.72, 0.96, 0.96 ,0.89, 0.93,
 0.71, 0.99, 0.99, 0.89, 0.88, 0.81, 1, 0.92, 0.77, 1])

with 0.3s 0.9020833333333332
with timepoint wise 500HZ 0.9291666666666666


0.9125

### SPD DA

In [71]:

n_cal = 4
n_class = 5
nb_part = len(participants)
SPD_accuracy_code_perso = np.zeros(nb_part)
SPD_tps_train_code_perso = np.zeros(nb_part)
SPD_tps_test_code_perso = np.zeros(nb_part)
SPD_accuracy_perso = np.zeros(nb_part)
nb_samples_windows = int((2.2-window_size)*n_class*n_cal*500)
history_list = []


for i in range(nb_part):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(nb_part) if j!=i]

    X_train = np.concatenate([X_riem[ind2take].reshape(-1,X_riem.shape[-2],X_riem.shape[-1]),X_riem[i][:nb_samples_windows]]).reshape(-1,X_riem.shape[-2],X_riem.shape[-1])
    Y_train = np.concatenate([y[ind2take].reshape(-1),y[i][:nb_samples_windows]]).reshape(-1)
    domains_train = np.concatenate([domains[ind2take].reshape(-1),domains[i][:nb_samples_windows]]).reshape(-1)
    X_test = X_riem[i][nb_samples_windows:]
    Y_test = y[i][nb_samples_windows:]
    labels_code_test = labels_code_list[i][n_cal*n_class:]

    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)
    # X_std = X_train.std(axis=0)
    # X_train /= X_std + 1e-8
    # X_std = X_test.std(axis=0)
    # X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train,Y_train,domains_train = balance(X_train,Y_train,domains_train)

    print("Creating the different pipelines")
    clf = SPDNet_Tensorflow(bimap_dims=[16,8,4])
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    batchsize = 64 #128 # 64 for burst
    epochs = 20 #45 # 20 for burst

    print("Fitting")
    start = time.time()
    lr = 1e-3
    weight_decay = 1e-4
    history_list.append(clf.fit(np.array(x_train), y_train,
                    batch_size=batchsize, epochs=epochs,
                    validation_data=(np.array(x_val), y_val), shuffle=True))
    SPD_tps_train_code_perso[i] = time.time() - start

    print("getting accuracy of participant ", i)
    start = time.time()
    y_pred = clf.predict(X_test)
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([1 if (y >= 0.5) else 0 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    SPD_accuracy_perso[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    labels_pred_accumul, _, mean_long_accumul = mpaa(
        y_pred_norm, codes, min_len=30, fps=500, consecutive=50, window_size=window_size
    )
    print("meannnnnn long",np.mean(mean_long_accumul))
    SPD_tps_test_code_perso[i] = time.time() - start
    SPD_accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(SPD_accuracy_perso)
print(SPD_tps_train_code_perso)
print(SPD_tps_test_code_perso)
print(SPD_accuracy_code_perso)
# pd.DataFrame(SPD_accuracy_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPD/score/DA_score.csv")
# pd.DataFrame(SPD_accuracy_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPD/score_code/DA_score_code.csv")
# pd.DataFrame(SPD_tps_train_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPD/temps_train_code/DA_tps_train_code.csv")
# pd.DataFrame(SPD_tps_test_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPD/temps_test_code/DA_tps_test_code.csv")


TL to the participant :  0
(1701375, 16, 16)
(1701375,)
(53625, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
2121/2121 [==============================] - 25s 11ms/step - loss: 0.4806 - sparse_categorical_accuracy: 0.7802 - val_loss: 0.4593 - val_sparse_categorical_accuracy: 0.7923
Epoch 2/20
2121/2121 [==============================] - 24s 11ms/step - loss: 0.4519 - sparse_categorical_accuracy: 0.7980 - val_loss: 0.4383 - val_sparse_categorical_accuracy: 0.8044
Epoch 3/20
2121/2121 [==============================] - 24s 11ms/step - loss: 0.4396 - sparse_categorical_accuracy: 0.8047 - val_loss: 0.4312 - val_sparse_categorical_accuracy: 0.8102
Epoch 4/20
2121/2121 [==============================] - 24s 11ms/step - loss: 0.4379 - sparse_categorical_accuracy: 0.8050 - val_loss: 0.4304 - val_sparse_categorical_accuracy: 0.8096
Epoch 5/20
2121/2121 [==============================] - 24s 11ms/step - loss: 0.4373 - sparse_categorical_accura

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.398888888888889
TL to the participant :  5
(1701375, 16, 16)
(1701375,)
(53625, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
2121/2121 [==============================] - 23s 11ms/step - loss: 0.4823 - sparse_categorical_accuracy: 0.7748 - val_loss: 0.4478 - val_sparse_categorical_accuracy: 0.7954
Epoch 2/20
2121/2121 [==============================] - 22s 11ms/step - loss: 0.4410 - sparse_categorical_accuracy: 0.8038 - val_loss: 0.4390 - val_sparse_categorical_accuracy: 0.8040
Epoch 3/20
2121/2121 [==============================] - 22s 11ms/step - loss: 0.4367 - sparse_categorical_accuracy: 0.8059 - val_loss: 0.4355 - val_sparse_categorical_accuracy: 0.8057
Epoch 4/20
2121/2121 [==============================] - 24s 11ms/step - loss: 0.4355 - sparse_categorical_accuracy: 0.8061 - val_loss: 0.4377 - val_sparse_categorical_accuracy: 0.8053
Epoch 5/20
2121/2121 [==============================] - 26s 12ms/step - loss: 0

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.4427083333333333
TL to the participant :  6
(1701375, 16, 16)
(1701375,)
(53625, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
2121/2121 [==============================] - 25s 11ms/step - loss: 0.4676 - sparse_categorical_accuracy: 0.7871 - val_loss: 0.4410 - val_sparse_categorical_accuracy: 0.8004
Epoch 2/20
2121/2121 [==============================] - 25s 12ms/step - loss: 0.4469 - sparse_categorical_accuracy: 0.8020 - val_loss: 0.4404 - val_sparse_categorical_accuracy: 0.8035
Epoch 3/20
2121/2121 [==============================] - 26s 12ms/step - loss: 0.4440 - sparse_categorical_accuracy: 0.8036 - val_loss: 0.4336 - val_sparse_categorical_accuracy: 0.8048
Epoch 4/20
2121/2121 [==============================] - 26s 12ms/step - loss: 0.4417 - sparse_categorical_accuracy: 0.8062 - val_loss: 0.4451 - val_sparse_categorical_accuracy: 0.7979
Epoch 5/20
2121/2121 [==============================] - 26s 12ms/step - loss: 

...

meannnnnn long 1.4379166666666665
TL to the participant :  19
(1701375, 16, 16)
(1701375,)
(53625, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
2121/2121 [==============================] - 31s 14ms/step - loss: 0.4643 - sparse_categorical_accuracy: 0.7906 - val_loss: 0.4431 - val_sparse_categorical_accuracy: 0.7974
Epoch 2/20
2121/2121 [==============================] - 30s 14ms/step - loss: 0.4385 - sparse_categorical_accuracy: 0.8041 - val_loss: 0.4266 - val_sparse_categorical_accuracy: 0.8092
Epoch 3/20
2121/2121 [==============================] - 30s 14ms/step - loss: 0.4344 - sparse_categorical_accuracy: 0.8068 - val_loss: 0.4265 - val_sparse_categorical_accuracy: 0.8108
Epoch 4/20
2121/2121 [==============================] - 30s 14ms/step - loss: 0.4334 - sparse_categorical_accuracy: 0.8074 - val_loss: 0.4458 - val_sparse_categorical_accuracy: 0.8014
Epoch 5/20
2121/2121 [==============================] - 30s 14ms/step - loss:

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.4162962962962964
TL to the participant :  20
(1701375, 16, 16)
(1701375,)
(53625, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
2121/2121 [==============================] - 30s 14ms/step - loss: 0.4716 - sparse_categorical_accuracy: 0.7839 - val_loss: 0.4600 - val_sparse_categorical_accuracy: 0.7927
Epoch 2/20
2121/2121 [==============================] - 29s 14ms/step - loss: 0.4490 - sparse_categorical_accuracy: 0.7983 - val_loss: 0.4624 - val_sparse_categorical_accuracy: 0.7850
Epoch 3/20
2121/2121 [==============================] - 29s 14ms/step - loss: 0.4466 - sparse_categorical_accuracy: 0.7994 - val_loss: 0.4463 - val_sparse_categorical_accuracy: 0.8014
Epoch 4/20
2121/2121 [==============================] - 29s 14ms/step - loss: 0.4446 - sparse_categorical_accuracy: 0.8001 - val_loss: 0.4434 - val_sparse_categorical_accuracy: 0.8010
Epoch 5/20
2121/2121 [==============================] - 29s 14ms/step - loss:

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.4352380952380952
TL to the participant :  22
(1701375, 16, 16)
(1701375,)
(53625, 16, 16)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1/20
2121/2121 [==============================] - 31s 14ms/step - loss: 0.4543 - sparse_categorical_accuracy: 0.7954 - val_loss: 0.4529 - val_sparse_categorical_accuracy: 0.7958
Epoch 2/20
2121/2121 [==============================] - 30s 14ms/step - loss: 0.4353 - sparse_categorical_accuracy: 0.8072 - val_loss: 0.4364 - val_sparse_categorical_accuracy: 0.8037
Epoch 3/20
2121/2121 [==============================] - 30s 14ms/step - loss: 0.4325 - sparse_categorical_accuracy: 0.8093 - val_loss: 0.4337 - val_sparse_categorical_accuracy: 0.8073
Epoch 4/20
2121/2121 [==============================] - 30s 14ms/step - loss: 0.4311 - sparse_categorical_accuracy: 0.8100 - val_loss: 0.4307 - val_sparse_categorical_accuracy: 0.8092
Epoch 5/20
2121/2121 [==============================] - 30s 14ms/step - loss:

In [1]:
print("with 0.3s",np.mean([1,  0.95, 0.8,  1,   0.96, 0.96, 0.89, 0.98, 0.91, 0.42, 0.96, 0.91, 0.93, 1,
 0.82, 0.91, 0.98, 0.85, 0.8 , 0.91, 1,   0.89, 0.84, 1  ]))
print("with timepoint wise 500HZ",np.mean([0.98,0.84,0.96,0.98,0.98,1.,  0.95,1.,  0.78 ,0.82,1.,0.91,0.91,0.96
,0.91,0.98,0.96,0.87,0.76,0.91,1.,  0.89,0.85,1. ]))
np.mean([1,0.96, 0.95, 0.98, 0.98, 1, 0.84, 0.93, 0.82, 0.64, 0.91, 0.85, 0.91, 0.98,
 0.85, 1, 1, 0.96, 0.87, 0.87, 0.98, 0.96, 0.82, 0.95])

NameError: name 'np' is not defined

### SPDBN SS

In [42]:
n_cal = 7
n_class = 5
nb_fold = 1
nb_part = 24
spdbn_accuracy_code_perso = np.zeros((nb_fold,nb_part))
spdbn_tps_train_code_perso = np.zeros((nb_fold,nb_part))
spdbn_tps_test_code_perso = np.zeros((nb_fold,nb_part))
spdbn_accuracy_perso = np.zeros((nb_fold,nb_part))

for k in range(nb_fold):
    for i in range(nb_part):
        print("TL to the participant : ", i)
        nb_sample_cal = int(n_class*n_cal*(2.2-window_size)*60)

        X_train = X[i][:nb_sample_cal]
        Y_train = y[i][:nb_sample_cal]
        X_test = X[i][nb_sample_cal:]
        Y_test = y[i][nb_sample_cal:]
        labels_code_test = labels_code_list[i][(n_class*n_cal):]

        print(X_train.shape)
        print(X_test.shape)
        # X_std = X_train.std(axis=0)
        # X_train /= X_std + 1e-8
        # X_std = X_test.std(axis=0)
        # X_test /= X_std + 1e-8

        print("balancing the number of ones and zeros")
        X_train, Y_train, domains_train = balance(X_train,Y_train,domains[i][:nb_sample_cal])

        print("Creating the different pipelines")
        lr = 1e-3
        # optimizer = riemannian_adam.RiemannianAdam(learning_rate=lr)
        batchsize = 64 #128 # 64 for burst
        epoch = 20 #45 # 20 for burst
        # clf = SPDNet_AJD(n_epochs=epoch,batch_size=batchsize,valid_split=0.1)
        clf = SPDNetBN_Module(bimap_dims=[32,28,14,7])

        print("Fitting")
        start = time.time()
        weight_decay = 1e-4
        
        x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)

        # Convert data into PyTorch tensors
        # X_train_tensor = torch.tensor(x_train, dtype=torch.float64)
        # y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        X_val_tensor = torch.tensor(x_val, dtype=torch.float64)
        y_val_tensor = torch.tensor(y_val, dtype=torch.long)
        # X_test_tensor = torch.tensor(X_test, dtype=torch.float64)
        # y_test_tensor = torch.tensor(Y_test, dtype=torch.long)

        # Create DataLoader for train, validation, and test sets
        # train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        # train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
        val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
        val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
        # test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
        # test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

        # Define loss function and optimizer
        criterion = torch.nn.CrossEntropyLoss()
        optimizer = torch_riemannian_adam.RiemannianAdam(clf.parameters(), lr=0.001)

        clf.fit(x_train,y_train)

        # Train the model
        # num_epochs = 20
        # for epoch in range(num_epochs):
        #     running_loss = 0.0
        #     for inputs, labels in train_dataloader:
        #         # Zero the parameter gradients
        #         optimizer.zero_grad()

        #         # Forward pass
        #         # print(inputs.shape)
        #         # print(labels.shape)
        #         outputs = clf(inputs)
        #         loss = criterion(outputs.float(), labels)

        #         # Backward pass and optimize
        #         loss.backward()
        #         optimizer.step()

        #         running_loss += loss.item()

        #     print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_dataloader)}")

        # print("Training finished!")
        spdbn_tps_train_code_perso[k][i] = time.time() - start

        # Validation
        clf.eval()
        val_correct = 0
        with torch.no_grad():
            for inputs, labels in val_dataloader:
                outputs = clf(inputs)
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == labels).sum().item()

        val_accuracy = val_correct / len(x_val)
        print(f"Validation Accuracy: {val_accuracy}")

        # Testing
        start = time.time()
        # test_correct = 0
        # y_pred= []
        # with torch.no_grad():
        #     for inputs, labels in test_dataloader:
        #         outputs = clf(inputs)
        #         _, predicted = torch.max(outputs, 1)
        #         y_pred.append(predicted)
        #         test_correct += (predicted == labels).sum().item()
                
        # test_accuracy = test_correct / len(X_test)
        y_pred = clf.predict(X_test)
        
        print("getting accuracy of participant ", i)
        # y_pred = np.concatenate(y_pred)
        y_pred = np.array(y_pred)
        y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
        y_test_norm = np.array([1 if (y >= 0.5) else 0 for y in Y_test])
        print(f"Test Accuracy: {balanced_accuracy_score(y_test_norm,y_pred_norm)}")

        tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
        spdbn_accuracy_perso[k][i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

        labels_pred_accumul, _, mean_long_accumul = mpaa(
            y_pred_norm, codes, min_len=30, fps=60, consecutive=50, window_size=window_size
        )
        spdbn_tps_test_code_perso[k][i] = time.time() - start
        spdbn_accuracy_code_perso[k][i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
        keras.backend.clear_session()

spdbn_accuracy_perso = np.mean(spdbn_accuracy_perso,axis=0)
spdbn_tps_train_code_perso = np.mean(spdbn_tps_train_code_perso,axis=0)
spdbn_tps_test_code_perso = np.mean(spdbn_tps_test_code_perso,axis=0)
spdbn_accuracy_code_perso = np.mean(spdbn_accuracy_code_perso,axis=0)

print(spdbn_accuracy_perso)
print(spdbn_tps_train_code_perso)
print(spdbn_tps_test_code_perso)
print(spdbn_accuracy_code_perso)
# pd.DataFrame(spdbn_accuracy_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPDBN/score/SS_score.csv")
# pd.DataFrame(spdbn_accuracy_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPDBN/score_code/SS_score_code.csv")
# pd.DataFrame(spdbn_tps_train_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPDBN/temps_train_code/SS_tps_train_code.csv")
# pd.DataFrame(spdbn_tps_test_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPDBN/temps_test_code/SS_tps_test_code.csv")

TL to the participant :  0
(3990, 32, 32)
(4560, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.6670618156592051
Epoch 2, Loss: 0.6376577218373617
Epoch 3, Loss: 0.6176846325397491
Epoch 4, Loss: 0.5814136366049448
Epoch 5, Loss: 0.5619668165842692
Epoch 6, Loss: 0.5250597645839056
Epoch 7, Loss: 0.5162107646465302
Epoch 8, Loss: 0.4504903306563695
Epoch 9, Loss: 0.4838244865338008
Epoch 10, Loss: 0.4249913344780604
Epoch 11, Loss: 0.3987770775953929
Epoch 12, Loss: 0.4462998112042745
Epoch 13, Loss: 0.36260150372982025
Epoch 14, Loss: 0.3328579639395078
Epoch 15, Loss: 0.3641149302323659
Epoch 16, Loss: 0.33864765365918476
Epoch 17, Loss: 0.3215875178575516
Epoch 18, Loss: 0.3105337123076121
Epoch 19, Loss: 0.30432985971371335
Epoch 20, Loss: 0.31580693026383716
Validation Accuracy: 0.4864864864864865
getting accuracy of participant  0
Test Accuracy: 0.5421203337104072


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  1
(3990, 32, 32)
(4560, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.6924769977728525
Epoch 2, Loss: 0.6832739611466726
Epoch 3, Loss: 0.6779614090919495
Epoch 4, Loss: 0.665468821922938
Epoch 5, Loss: 0.6554269989331564
Epoch 6, Loss: 0.6437441408634186
Epoch 7, Loss: 0.6249582668145498
Epoch 8, Loss: 0.608268012603124
Epoch 9, Loss: 0.5883064866065979
Epoch 10, Loss: 0.5593211352825165
Epoch 11, Loss: 0.5167825818061829
Epoch 12, Loss: 0.5231404105822245
Epoch 13, Loss: 0.47932080427805585
Epoch 14, Loss: 0.47347629566987354
Epoch 15, Loss: 0.44129058718681335
Epoch 16, Loss: 0.43466027081012726
Epoch 17, Loss: 0.41506027181943256
Epoch 18, Loss: 0.37452515214681625
Epoch 19, Loss: 0.39177773396174115
Epoch 20, Loss: 0.3533770640691121
Validation Accuracy: 0.7567567567567568
getting accuracy of participant  1
Test Accuracy: 0.6083763433257918


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  2
(3990, 32, 32)
(4560, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.7005536953608195
Epoch 2, Loss: 0.7020728985468546
Epoch 3, Loss: 0.6691405475139618
Epoch 4, Loss: 0.653730571269989
Epoch 5, Loss: 0.6470721960067749
Epoch 6, Loss: 0.6161286234855652
Epoch 7, Loss: 0.6374632815519968
Epoch 8, Loss: 0.6339979767799377
Epoch 9, Loss: 0.6328570942083994
Epoch 10, Loss: 0.595938632885615
Epoch 11, Loss: 0.5801632503668467
Epoch 12, Loss: 0.5921085377534231
Epoch 13, Loss: 0.5850401918093363
Epoch 14, Loss: 0.5756156543890635
Epoch 15, Loss: 0.569974958896637
Epoch 16, Loss: 0.5381475587685903
Epoch 17, Loss: 0.5520534366369247
Epoch 18, Loss: 0.5388025641441345
Epoch 19, Loss: 0.5198117047548294
Epoch 20, Loss: 0.5436254243055979
Validation Accuracy: 0.8108108108108109
getting accuracy of participant  2
Test Accuracy: 0.7242912188914028


...

TL to the participant :  22
(3990, 32, 32)
(4560, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.7380663653214773
Epoch 2, Loss: 0.7275590896606445
Epoch 3, Loss: 0.7220149437586466
Epoch 4, Loss: 0.7005257109800974
Epoch 5, Loss: 0.7039543489615122
Epoch 6, Loss: 0.6819550593694051
Epoch 7, Loss: 0.6619764467080435
Epoch 8, Loss: 0.6589224139849345
Epoch 9, Loss: 0.6802191535631815
Epoch 10, Loss: 0.6320760448773702
Epoch 11, Loss: 0.6269224584102631
Epoch 12, Loss: 0.6037481526533762
Epoch 13, Loss: 0.6034643650054932
Epoch 14, Loss: 0.5901906788349152
Epoch 15, Loss: 0.6035201847553253
Epoch 16, Loss: 0.5842929383118948
Epoch 17, Loss: 0.576670785744985
Epoch 18, Loss: 0.573326845963796
Epoch 19, Loss: 0.5562935620546341
Epoch 20, Loss: 0.6080338060855865
Validation Accuracy: 0.6216216216216216
getting accuracy of participant  22
Test Accuracy: 0.5454079468325792


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


TL to the participant :  23
(3990, 32, 32)
(4560, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.6831793189048767
Epoch 2, Loss: 0.6435909271240234
Epoch 3, Loss: 0.6125143865744272
Epoch 4, Loss: 0.5654971301555634
Epoch 5, Loss: 0.5265133529901505
Epoch 6, Loss: 0.49308545390764874
Epoch 7, Loss: 0.4422021806240082
Epoch 8, Loss: 0.4443596452474594
Epoch 9, Loss: 0.4269181787967682
Epoch 10, Loss: 0.4194196214278539
Epoch 11, Loss: 0.4048919677734375
Epoch 12, Loss: 0.3777568240960439
Epoch 13, Loss: 0.3820605476697286
Epoch 14, Loss: 0.3731785515944163
Epoch 15, Loss: 0.41055944561958313
Epoch 16, Loss: 0.3706581691900889
Epoch 17, Loss: 0.40110041201114655
Epoch 18, Loss: 0.3501882354418437
Epoch 19, Loss: 0.36358340084552765
Epoch 20, Loss: 0.33483507732550305
Validation Accuracy: 0.5945945945945946
getting accuracy of participant  23
Test Accuracy: 0.537657310520362


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[0.54212033 0.60837634 0.72429122 0.53147094 0.69396034 0.86953832
 0.6473593  0.69875919 0.55674668 0.69970482 0.77710513 0.64896776
 0.70833923 0.6197681  0.65695701 0.50480769 0.50938561 0.61142534
 0.72607643 0.68525523 0.62991374 0.75867859 0.54540795 0.53765731]
[18.44663453 16.28622532 15.78565598 15.71362638 15.67448139 15.76868176
 15.55956984 15.57822871 15.52039957 15.45689154 17.08833385 15.49804115
 16.24478412 14.94273806 16.7355566  16.01483178 16.20480704 15.73924708
 15.38939142 15.31289315 15.20398235 15.21854758 15.21695781 15.16730237]
[5.72324562 5.43994999 5.30060172 5.23392653 5.22808695 5.19848204
 5.26955986 5.20366883 5.25586033 5.21971464 5.74662328 5.60907698
 5.40477943 5.28498769 5.68034101 5.38720512 5.24848819 5.14650464
 5.17179084 5.19780469 5.09268212 5.14444709 5.12405944 5.00490522]
[0.42 0.8  0.95 0.38 0.9  1.   0.75 0.88 0.6  0.68 0.95 0.8  0.95 0.75
 0.68 0.2  0.25 0.65 0.82 0.88 0.88 0.85 0.38 0.45]


In [45]:
print("with 0.3s",np.mean([0.42, 0.8,  0.95, 0.38, 0.9,  1,   0.75, 0.88, 0.6,  0.68, 0.95, 0.8,  0.95, 0.75,
 0.68, 0.2 , 0.25, 0.65, 0.82, 0.88, 0.88, 0.85, 0.38 ,0.45]))
print("with 0.25s",np.mean([0.52, 0.98, 0.9,  0.92, 0.85, 0.9,  0.62, 0.92, 0.75, 0.57, 0.95, 0.82, 0.8,  0.92,
 0.57, 0.78, 0.98, 0.92, 0.68, 0.7,  0.98, 0.92, 0.72, 0.98]))

with 0.3s 0.7020833333333334
with 0.3s 0.8187500000000001


### SPDBN DA

In [46]:
n_cal = 4
n_class = 5
nb_part = len(participants)
SPDBN_accuracy_code_perso = np.zeros(nb_part)
SPDBN_tps_train_code_perso = np.zeros(nb_part)
SPDBN_tps_test_code_perso = np.zeros(nb_part)
SPDBN_accuracy_perso = np.zeros(nb_part)
nb_samples_windows = int((2.2-window_size)*n_class*n_cal*60)
history_list = []


for i in range(nb_part):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(nb_part) if j!=i]
    nb_sample_cal = int(n_class*n_cal*(2.2-window_size)*60)

    X_train = np.concatenate([np.concatenate(X[ind2take]).reshape(-1,X.shape[-2],X.shape[-1]),X[i][:nb_sample_cal]]).reshape(-1,X.shape[-2],X.shape[-1])
    Y_train = np.concatenate([np.concatenate(Y[ind2take]).reshape(-1),Y[i][:nb_sample_cal]]).reshape(-1)
    domains_train = np.concatenate([np.concatenate(domains[ind2take]).reshape(-1),domains[i][:nb_sample_cal]]).reshape(-1)

    X_test = X[i][nb_sample_cal:]
    Y_test = y[i][nb_sample_cal:]
    labels_code_test = labels_code_list[i][(n_class*n_cal):]

    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)
    # X_std = X_train.std(axis=0)
    # X_train /= X_std + 1e-8
    # X_std = X_test.std(axis=0)
    # X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train,Y_train,domains_train = balance(X_train,Y_train,domains_train)

    print("Creating the different pipelines")
    clf = SPDNetBN_Module(bimap_dims=[32,28,14,7])
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    batchsize = 64 #128 # 64 for burst
    epochs = 20 #45 # 20 for burst

    print("Fitting")
    start = time.time()
    lr = 1e-3
    weight_decay = 1e-4
    clf.fit(np.array(x_train), y_train,
                    batch_size=batchsize, epochs=epochs, shuffle=True)
    SPDBN_tps_train_code_perso[i] = time.time() - start

    print("getting accuracy of participant ", i)
    start = time.time()
    y_pred = clf.predict(X_test)
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([1 if (y >= 0.5) else 0 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    SPDBN_accuracy_perso[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    labels_pred_accumul, _, mean_long_accumul = mpaa(
        y_pred_norm, codes, min_len=30, fps=60, consecutive=50, window_size=window_size
    )
    print("meannnnnn long",np.mean(mean_long_accumul))
    SPDBN_tps_test_code_perso[i] = time.time() - start
    SPDBN_accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(SPDBN_accuracy_perso)
print(SPDBN_tps_train_code_perso)
print(SPDBN_tps_test_code_perso)
print(SPDBN_accuracy_code_perso)
# pd.DataFrame(SPDBN_accuracy_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPDBN/score/DA_score.csv")
# pd.DataFrame(SPDBN_accuracy_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPDBN/score_code/DA_score_code.csv")
# pd.DataFrame(SPDBN_tps_train_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPDBN/temps_train_code/DA_tps_train_code.csv")
# pd.DataFrame(SPDBN_tps_test_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPDBN/temps_test_code/DA_tps_test_code.csv")


TL to the participant :  0
(198930, 32, 32)
(198930,)
(6270, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5417503410717472
Epoch 2, Loss: 0.4748594481498003
Epoch 3, Loss: 0.4658820100594312
Epoch 4, Loss: 0.4637272903928533
Epoch 5, Loss: 0.44802139222156256
Epoch 6, Loss: 0.441924013372045
Epoch 7, Loss: 0.4406695463694632
Epoch 8, Loss: 0.43488525645807385
Epoch 9, Loss: 0.43567147210706025
Epoch 10, Loss: 0.4336367679061368
Epoch 11, Loss: 0.4310180840548128
Epoch 12, Loss: 0.4312273330288008
Epoch 13, Loss: 0.4313997090794146
Epoch 14, Loss: 0.4285330147249624
Epoch 15, Loss: 0.42750218213768676
Epoch 16, Loss: 0.4282956763636321
Epoch 17, Loss: 0.4287677133688703
Epoch 18, Loss: 0.4256867481744848
Epoch 19, Loss: 0.4253149752621539
Epoch 20, Loss: 0.4271982639329508
getting accuracy of participant  0


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.369444444444444
TL to the participant :  1
(198930, 32, 32)
(198930,)
(6270, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5853809227701277
Epoch 2, Loss: 0.48300125810783356
Epoch 3, Loss: 0.457795417169109
Epoch 4, Loss: 0.4488656931789592
Epoch 5, Loss: 0.4407566257286817
Epoch 6, Loss: 0.4360110773704946
Epoch 7, Loss: 0.43673547834623605
Epoch 8, Loss: 0.43018756178207695
Epoch 9, Loss: 0.4318208107724786
Epoch 10, Loss: 0.42820839735213667
Epoch 11, Loss: 0.4262898521265015
Epoch 12, Loss: 0.42668063729070127
Epoch 13, Loss: 0.4264328798162751
Epoch 14, Loss: 0.42601174395531416
Epoch 15, Loss: 0.42599860834889114
Epoch 16, Loss: 0.42660940915811807
Epoch 17, Loss: 0.42779949493706226
Epoch 18, Loss: 0.42532186349853873
Epoch 19, Loss: 0.424868002475705
Epoch 20, Loss: 0.4228742041741498
getting accuracy of participant  1
meannnnnn long 1.4011904761904763
TL to the participant :  2
(198930, 32, 32)
(19893

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.4373188405797104
TL to the participant :  3
(198930, 32, 32)
(198930,)
(6270, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5548694466706365
Epoch 2, Loss: 0.4486298034898937
Epoch 3, Loss: 0.45020582620054483
Epoch 4, Loss: 0.46375483518932015
Epoch 5, Loss: 0.43712496431544423
Epoch 6, Loss: 0.43578824505675584
Epoch 7, Loss: 0.43229020276339725
Epoch 8, Loss: 0.4392341764178127
Epoch 9, Loss: 0.4353236753959209
Epoch 10, Loss: 0.4273193086264655
Epoch 11, Loss: 0.4280849650967866
Epoch 12, Loss: 0.4268827982014045
Epoch 13, Loss: 0.427269849460572
Epoch 14, Loss: 0.4226973226759583
Epoch 15, Loss: 0.4236743545625359
Epoch 16, Loss: 0.42516377789434046
Epoch 17, Loss: 0.4224509425694123
Epoch 18, Loss: 0.4231025844346732
Epoch 19, Loss: 0.4203831149498001
Epoch 20, Loss: 0.4230143786408007
getting accuracy of participant  3


...

meannnnnn long 1.3922764227642277
TL to the participant :  20
(198930, 32, 32)
(198930,)
(6270, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5473017134936526
Epoch 2, Loss: 0.4629300682572648
Epoch 3, Loss: 0.4567105791065842
Epoch 4, Loss: 0.4528674747562036
Epoch 5, Loss: 0.4478533180663362
Epoch 6, Loss: 0.4456899323267862
Epoch 7, Loss: 0.44079405907541513
Epoch 8, Loss: 0.4363456925493665
Epoch 9, Loss: 0.4392072050832212
Epoch 10, Loss: 0.4354856200516224
Epoch 11, Loss: 0.43449071829672903
Epoch 12, Loss: 0.44120132620446384
Epoch 13, Loss: 0.4342706841416657
Epoch 14, Loss: 0.43505042261676863
Epoch 15, Loss: 0.4321113673504442
Epoch 16, Loss: 0.43041296885348856
Epoch 17, Loss: 0.43086026603123173
Epoch 18, Loss: 0.4359815197531134
Epoch 19, Loss: 0.4302868451923132
Epoch 20, Loss: 0.42817777761956677
getting accuracy of participant  20


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.3817901234567895
TL to the participant :  21
(198930, 32, 32)
(198930,)
(6270, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5423472343245521
Epoch 2, Loss: 0.4578027534298599
Epoch 3, Loss: 0.4457512431545183
Epoch 4, Loss: 0.4395236726850271
Epoch 5, Loss: 0.43695563985966146
Epoch 6, Loss: 0.43160005018580705
Epoch 7, Loss: 0.43112553108949214
Epoch 8, Loss: 0.43104058504104614
Epoch 9, Loss: 0.42957032535923645
Epoch 10, Loss: 0.426525937858969
Epoch 11, Loss: 0.4301814802456647
Epoch 12, Loss: 0.4263255539117381
Epoch 13, Loss: 0.4248453834443353
Epoch 14, Loss: 0.42319280782248825
Epoch 15, Loss: 0.42223922337871045
Epoch 16, Loss: 0.4225134286098182
Epoch 17, Loss: 0.42440417752368376
Epoch 18, Loss: 0.42396262526744977
Epoch 19, Loss: 0.42329002264887094
Epoch 20, Loss: 0.4244750360958278
getting accuracy of participant  21


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.4059829059829063
TL to the participant :  22
(198930, 32, 32)
(198930,)
(6270, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5455517984228209
Epoch 2, Loss: 0.46839966834522784
Epoch 3, Loss: 0.43579488882096484
Epoch 4, Loss: 0.4260140161495656
Epoch 5, Loss: 0.42807291098870337
Epoch 6, Loss: 0.4235118117649108
Epoch 7, Loss: 0.424943171441555
Epoch 8, Loss: 0.42385562614072114
Epoch 9, Loss: 0.4201002477784641
Epoch 10, Loss: 0.41764768259599805
Epoch 11, Loss: 0.42008716822601855
Epoch 12, Loss: 0.41822978877462447
Epoch 13, Loss: 0.4161361383739859
Epoch 14, Loss: 0.42046002845745534
Epoch 15, Loss: 0.4178725676611066
Epoch 16, Loss: 0.41497914906358346
Epoch 17, Loss: 0.41528532176744193
Epoch 18, Loss: 0.41715532913804054
Epoch 19, Loss: 0.416489124414511
Epoch 20, Loss: 0.41262819652911276
getting accuracy of participant  22
meannnnnn long 1.4409523809523812
TL to the participant :  23
(198930, 32, 32)


In [16]:
print("with 0.3s",np.mean([1,   0.95, 0.95, 0.56, 1,   0.95, 0.93 ,0.85 ,0.78, 0.2,  0.98, 0.89, 0.87, 0.96,
 0.58, 0.91, 1,  0.95, 0.85, 0.93, 1,  0.85, 0.85, 0.93]))
print("with 0.25s",np.mean([1, 0.98, 0.93, 0.91, 0.98, 0.95, 0.87, 0.93, 0.87, 0.76, 0.98, 0.87 ,0.91, 0.96,
 0.69 ,0.78, 0.98, 0.95, 0.84 ,0.69 ,0.96, 0.96, 0.69, 0.98]))

with 0.3s 0.8633333333333333
with 0.25s 0.8925000000000001


### SPDBN DG

In [48]:
n_cal = 4
n_class = 5
nb_part = len(participants)
SPDBN_accuracy_code_perso = np.zeros(nb_part)
SPDBN_tps_train_code_perso = np.zeros(nb_part)
SPDBN_tps_test_code_perso = np.zeros(nb_part)
SPDBN_accuracy_perso = np.zeros(nb_part)
nb_samples_windows = int((2.2-window_size)*n_class*n_cal*60)
history_list = []


for i in range(nb_part):
    print("TL to the participant : ", i)
    ind2take = [j for j in range(nb_part) if j!=i]

    X_train = np.concatenate(X[ind2take]).reshape(-1,X.shape[-2],X.shape[-1])
    Y_train = np.concatenate(y[ind2take]).reshape(-1)
    domains_train = np.concatenate(domains[ind2take]).reshape(-1)
    X_test = X[i]
    Y_test = y[i]
    labels_code_test = labels_code_list[i]

    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)
    # X_std = X_train.std(axis=0)
    # X_train /= X_std + 1e-8
    # X_std = X_test.std(axis=0)
    # X_test /= X_std + 1e-8

    print("balancing the number of ones and zeros")
    X_train,Y_train,domains_train = balance(X_train,Y_train,domains_train)

    print("Creating the different pipelines")
    clf = SPDNetBN_Module(bimap_dims=[32,28,14,7])
    x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42, shuffle=True)
    batchsize = 64 #128 # 64 for burst
    epochs = 20 #45 # 20 for burst

    print("Fitting")
    start = time.time()
    lr = 1e-3
    weight_decay = 1e-4
    clf.fit(np.array(x_train), y_train,
                    batch_size=batchsize, epochs=epochs, shuffle=True)
    SPDBN_tps_train_code_perso[i] = time.time() - start

    print("getting accuracy of participant ", i)
    start = time.time()
    y_pred = clf.predict(X_test)
    y_pred = np.array(y_pred)
    y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
    y_test_norm = np.array([1 if (y >= 0.5) else 0 for y in Y_test])

    tn, fp, fn, tp = confusion_matrix(y_test_norm, y_pred_norm).ravel()
    SPDBN_accuracy_perso[i] = balanced_accuracy_score(y_test_norm,y_pred_norm)

    labels_pred_accumul, _, mean_long_accumul = mpaa(
        y_pred_norm, codes, min_len=30, fps=60, consecutive=50, window_size=window_size
    )
    print("meannnnnn long",np.mean(mean_long_accumul))
    SPDBN_tps_test_code_perso[i] = time.time() - start
    SPDBN_accuracy_code_perso[i] = np.round(accuracy_score(labels_code_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    keras.backend.clear_session()

print(SPDBN_accuracy_perso)
print(SPDBN_tps_train_code_perso)
print(SPDBN_tps_test_code_perso)
print(SPDBN_accuracy_code_perso)
# pd.DataFrame(SPDBN_accuracy_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPDBN/score/DG_score.csv")
# pd.DataFrame(SPDBN_accuracy_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPDBN/score_code/DG_score_code.csv")
# pd.DataFrame(SPDBN_tps_train_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPDBN/temps_train_code/DG_tps_train_code.csv")
# pd.DataFrame(SPDBN_tps_test_code_perso).to_csv("C:/Users/s.velut/Documents/These/Protheus_PHD/results/new_dataset/Score_TF/SPDBN/temps_test_code/DG_tps_test_code.csv")


TL to the participant :  0
(196650, 32, 32)
(196650,)
(8550, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5445233749542312
Epoch 2, Loss: 0.45403583023859106
Epoch 3, Loss: 0.44354211966039636
Epoch 4, Loss: 0.4391560049160667
Epoch 5, Loss: 0.4507721551793366
Epoch 6, Loss: 0.4606000963642663
Epoch 7, Loss: 0.4341570460513646
Epoch 8, Loss: 0.4333368237664106
Epoch 9, Loss: 0.43158820471744763
Epoch 10, Loss: 0.4351811825993504
Epoch 11, Loss: 0.42981304175297735
Epoch 12, Loss: 0.4336275132041675
Epoch 13, Loss: 0.43312833137191803
Epoch 14, Loss: 0.4342232625710634
Epoch 15, Loss: 0.429224753568295
Epoch 16, Loss: 0.4314336686030678
Epoch 17, Loss: 0.43290879709918506
Epoch 18, Loss: 0.43147510190019495
Epoch 19, Loss: 0.42978068726807245
Epoch 20, Loss: 0.4269242602374714
getting accuracy of participant  0


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.3595555555555554
TL to the participant :  1
(196650, 32, 32)
(196650,)
(8550, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5367811607277911
Epoch 2, Loss: 0.44576072268806427
Epoch 3, Loss: 0.43451352435138385
Epoch 4, Loss: 0.4320426723467031
Epoch 5, Loss: 0.42791051355746423
Epoch 6, Loss: 0.42685331386539777
Epoch 7, Loss: 0.4253420542351342
Epoch 8, Loss: 0.42416984702758637
Epoch 9, Loss: 0.4229057174426294
Epoch 10, Loss: 0.42112698974345514
Epoch 11, Loss: 0.4291564924090276
Epoch 12, Loss: 0.42176262256891833
Epoch 13, Loss: 0.4220528003608756
Epoch 14, Loss: 0.42032607789096155
Epoch 15, Loss: 0.4245030457087656
Epoch 16, Loss: 0.420563055298074
Epoch 17, Loss: 0.41810995964664716
Epoch 18, Loss: 0.419187248342122
Epoch 19, Loss: 0.41706249612593366
Epoch 20, Loss: 0.4143006270935413
getting accuracy of participant  1
meannnnnn long 1.4677083333333334
TL to the participant :  2
(196650, 32, 32)
(1966

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.4563888888888885
TL to the participant :  3
(196650, 32, 32)
(196650,)
(8550, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5664927894654481
Epoch 2, Loss: 0.45773267015638086
Epoch 3, Loss: 0.4416050899170133
Epoch 4, Loss: 0.4366694384883986
Epoch 5, Loss: 0.4329181052008165
Epoch 6, Loss: 0.432305971977739
Epoch 7, Loss: 0.4300156691564402
Epoch 8, Loss: 0.4306813112714074
Epoch 9, Loss: 0.4312842794557805
Epoch 10, Loss: 0.42828612259254156
Epoch 11, Loss: 0.42419561832083075
Epoch 12, Loss: 0.4298089576568528
Epoch 13, Loss: 0.42418051354969916
Epoch 14, Loss: 0.4259842431592376
Epoch 15, Loss: 0.4275863626493296
Epoch 16, Loss: 0.4298979094023761
Epoch 17, Loss: 0.42506367309762555
Epoch 18, Loss: 0.424425869708947
Epoch 19, Loss: 0.42415308822756226
Epoch 20, Loss: 0.4226901144849453
getting accuracy of participant  3


...

meannnnnn long 1.4409420289855073
TL to the participant :  18
(196650, 32, 32)
(196650,)
(8550, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5375700834475958
Epoch 2, Loss: 0.44762688757402624
Epoch 3, Loss: 0.43208522511565167
Epoch 4, Loss: 0.4317995311007669
Epoch 5, Loss: 0.4236834145817361
Epoch 6, Loss: 0.4237237009606343
Epoch 7, Loss: 0.42641155027118127
Epoch 8, Loss: 0.4239568754500551
Epoch 9, Loss: 0.4202112571995249
Epoch 10, Loss: 0.4193235536809024
Epoch 11, Loss: 0.42046215553057525
Epoch 12, Loss: 0.4195119903492833
Epoch 13, Loss: 0.4147660775853711
Epoch 14, Loss: 0.41917242975574237
Epoch 15, Loss: 0.4173407104646735
Epoch 16, Loss: 0.41885123929016205
Epoch 17, Loss: 0.4146329944312808
Epoch 18, Loss: 0.4144564953010544
Epoch 19, Loss: 0.4173791502186432
Epoch 20, Loss: 0.41624880767622485
getting accuracy of participant  18
meannnnnn long 1.4545045045045046
TL to the participant :  19
(196650, 32, 32)
(19

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.4722222222222223
TL to the participant :  20
(196650, 32, 32)
(196650,)
(8550, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.633147238153714
Epoch 2, Loss: 0.5016833457786575
Epoch 3, Loss: 0.46993258560127893
Epoch 4, Loss: 0.45968460176773224
Epoch 5, Loss: 0.45521303787533
Epoch 6, Loss: 0.447364296719962
Epoch 7, Loss: 0.44628304349103937
Epoch 8, Loss: 0.43948003855854156
Epoch 9, Loss: 0.439770209930631
Epoch 10, Loss: 0.437536484639164
Epoch 11, Loss: 0.4368320697616683
Epoch 12, Loss: 0.4347381031089149
Epoch 13, Loss: 0.43659509276684094
Epoch 14, Loss: 0.43459114997754456
Epoch 15, Loss: 0.4327034848363032
Epoch 16, Loss: 0.43271684505251556
Epoch 17, Loss: 0.43526457326685486
Epoch 18, Loss: 0.4347704769123213
Epoch 19, Loss: 0.43559354602583783
Epoch 20, Loss: 0.4341752656363687
getting accuracy of participant  20
meannnnnn long 1.398507462686567
TL to the participant :  21
(196650, 32, 32)
(196650,

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


meannnnnn long 1.4263636363636358
TL to the participant :  22
(196650, 32, 32)
(196650,)
(8550, 32, 32)
balancing the number of ones and zeros
Creating the different pipelines
Fitting
Epoch 1, Loss: 0.5418547660465768
Epoch 2, Loss: 0.460513880837105
Epoch 3, Loss: 0.4495846812904117
Epoch 4, Loss: 0.4450697929491639
Epoch 5, Loss: 0.44242744295022235
Epoch 6, Loss: 0.4388654462904798
Epoch 7, Loss: 0.43752894700751477
Epoch 8, Loss: 0.4371370984160382
Epoch 9, Loss: 0.43354950032450934
Epoch 10, Loss: 0.42918288578157837
Epoch 11, Loss: 0.4289782811766085
Epoch 12, Loss: 0.4284539773412373
Epoch 13, Loss: 0.42704761075408093
Epoch 14, Loss: 0.42509769692486926
Epoch 15, Loss: 0.42293265532599134
Epoch 16, Loss: 0.4224995953056652
Epoch 17, Loss: 0.4230427059967056
Epoch 18, Loss: 0.4211736549973017
Epoch 19, Loss: 0.419315462880455
Epoch 20, Loss: 0.4201817811713388
getting accuracy of participant  22
meannnnnn long 1.4421985815602836
TL to the participant :  23
(196650, 32, 32)
(1966

In [19]:
print("with 0.3s",np.mean([1, 0.81 ,0.91, 0.99 ,0.93 ,0.96 ,0.85 ,0.88, 0.84, 0.32, 0.49, 0.89 ,0.92, 0.95,
 0.85, 0.95, 0.95, 0.89, 0.76, 0.91, 0.97 ,0.88, 0.8,  0.97]))
print("with 0.25s",np.mean([1, 0.93, 0.92, 0.99, 0.95, 1 ,0.91, 0.92, 0.75, 0.64, 0.96, 0.88, 0.84, 0.93,
 0.8,  0.91, 0.99 ,0.87, 0.85, 0.85, 0.99, 0.92, 0.63, 1 ]))

with 0.3s 0.8612500000000001
with 0.25s 0.8929166666666667


# Old test

### Pick an architecture

In [ ]:
#  clf = basearchi_patchembedding(windows_size = n_samples_windows, n_channel_input = n_channels) # other stim
# clf = basearchi_patchembeddingdilation(windows_size = n_samples_windows, n_channel_input = n_channels) # burst
# clf = vanilliaEEG2Code(windows_size = n_samples_windows, n_channel_input = n_channels) # burst
# clf = trueVanilliaEEG2Code(windows_size = n_samples_windows, n_channel_input = n_channels) # burst
# clf = basearchitest(windows_size = n_samples_windows, n_channel_input = n_channels)
#c lf = basearchitest_batchnorm(windows_size = n_samples_windows, n_channel_input = n_channels)
# clf = EEGnet_Inception(windows_size = n_samples_windows, n_channel_input = n_channels)
# clf = basearchi(windows_size = n_samples_windows, n_channel_input = n_channels)

clf = make_pipeline(XdawnCovariances(nfilter=8, estimator="oas", xdawn_estimator="lwf"),MDM())
clf2 = make_pipeline(XdawnCovariances(nfilter=8, estimator="lwf", xdawn_estimator="lwf"),
    TangentSpace(),
    LDA(solver="lsqr", shrinkage="auto"))
clf3 = make_pipeline(XdawnCovariances(nfilter=8, estimator="oas", xdawn_estimator="lwf"),
    TangentSpace(),
    svm.SVC())
clf4 = make_pipeline(Xdawn(nfilter=8, estimator="lwf"),Vectorizer(),LDA(solver="lsqr", shrinkage="auto"))
# clf5 = basearchi(windows_size = n_samples_windows, n_channel_input = n_channels)


# xdawn = XdawnCovariances(nfilter=3, estimator="lwf", xdawn_estimator="scm")
# clf = MDM()


# clf.steps

In [ ]:
i=2
n_cal = 7
n_class = 5
window_size = 0.25
nb_samples_windows = int((2.2-window_size)*n_cal*n_class*60)
X_train = X[i][:nb_samples_windows]
Y_train = Y[i][:nb_samples_windows]
domains_train = domains[i][:nb_samples_windows]
X_test = X[i][nb_samples_windows:]
Y_test = Y[i][nb_samples_windows:]
labels_code_test = labels_code_list[i][n_cal*n_class:]

X_std = X_train.std(axis=0)
X_train /= X_std + 1e-8
X_std = X_test.std(axis=0)
X_test /= X_std + 1e-8

rus = RandomUnderSampler()
counter=np.array(range(0,len(Y_train))).reshape(-1,1)
index,_ = rus.fit_resample(counter,Y_train[:])
X_train = np.squeeze(X_train[index,:,:], axis=1)
Y_train = np.squeeze(Y_train[index])
# rus = RandomUnderSampler()
# counter=np.array(range(0,len(Y_test))).reshape(-1,1)
# index,_ = rus.fit_resample(counter,Y_test[:])
# X_test = np.squeeze(X_test[index,:,:], axis=1)
# Y_test = np.squeeze(Y_test[index])

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print(nb_samples_windows)



(364, 8, 125)
(364,)
(4680, 8, 125)
(4680,)
4095


### Cut the train in train and valid
Also set some HP of the network

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=42, shuffle=True)
batchsize = 128 #128 # 64 for burst
epochs = 45 #45 # 20 for burst

### Attach an optimizer and train the network

res_X_train=[]
for i in range(len(X_train)):
    res_X_train.append(np.array(X_train[i:i+1]).reshape(n_samples_windows, n_channels, 1))
res_X_val=[]
for i in range(len(x_val)):
    res_X_val.append(np.array(x_val[i:i+1]).reshape(n_samples_windows, n_channels, 1))

np.array(res_X_val).shape

np.array(res_X_train).shape

In [ ]:
# lr = 1e-3
# weight_decay = 1e-4
# optimizer = keras.optimizers.Adam(learning_rate=lr, amsgrad=True)
# clf5.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
# X_trainp = xdawn.fit_transform(X_train,y_train)
# X_valp = xdawn.fit_transform(x_val,y_val)
# X_testp = xdawn.fit_transform(X_test,Y_test)>
history = clf.fit(np.array(x_train), y_train)
history2 = clf2.fit(np.array(x_train), y_train)
history3 = clf3.fit(np.array(x_train), y_train)
history4 = clf4.fit(np.array(x_train), y_train)

history_list = [history,history2,history3,history4]

reg_x_train = x_train.reshape(x_train.shape[0], x_train.shape[1] * x_train.shape[2])
# history_r1 = rm.fit(np.array(reg_x_train), y_train)
keras.backend.clear_session()


In [ ]:
# rm.score(X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2]),Y_test)
# pred = rm.predict(X_test.reshape(X_test.shape[0], X_test.shape[1] * X_test.shape[2]))
# y_pred = [1 if (Y >= 0.5) else 0 for Y in pred]
# print(np.count_nonzero(y_pred==Y_test)/len(Y_test))

In [ ]:
from sklearn.metrics import balanced_accuracy_score


print("RG")
print(balanced_accuracy_score(y_train,history.predict(x_train)))
print(balanced_accuracy_score(y_val,history.predict(x_val)))
print(balanced_accuracy_score(Y_test,history.predict(X_test)))

print("RG+LDA")
print(balanced_accuracy_score(y_train,history2.predict(x_train)))
print(balanced_accuracy_score(y_val,history2.predict(x_val)))
print(balanced_accuracy_score(Y_test,history2.predict(X_test)))

print("RG+SVC")
print(balanced_accuracy_score(y_train,history3.predict(x_train)))
print(balanced_accuracy_score(y_val,history3.predict(x_val)))
print(balanced_accuracy_score(Y_test,history3.predict(X_test)))

print("Xdw+LDA")
print(balanced_accuracy_score(y_train,history4.predict(x_train)))
print(balanced_accuracy_score(y_val,history4.predict(x_val)))
print(balanced_accuracy_score(Y_test,history4.predict(X_test)))


RG
0.5291922531884743
0.5484234234234234
0.5054785330948122
RG+LDA
0.8179263108171941
0.46621621621621623
0.5024038461538461
RG+SVC
0.7182569674067076
0.47935435435435436
0.509559481216458
Xdw+LDA
0.7423948984411903
0.5352852852852853
0.5254919499105546


### Model and accuracy and loss
Just check that the model learnt something

In [ ]:
p=2
plt.figure(figsize=(12,12))
# summarize history for accuracy
plt.subplot(2,1,1)
plt.plot(history_list[0].history['accuracy'])
plt.plot(history_list[0].history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
# summarize history for loss
plt.subplot(2,1,2)
plt.plot(history_list[p].history['loss'])
plt.plot(history_list[p].history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
print()
plt.show()


AttributeError: 'Pipeline' object has no attribute 'history'

filename = ''
np.save(filename + "std_from_calibration", X_std)
model_filename = os.path.join(os.getcwd(), filename + '0.7' + "trainedmodel")
# Save the model if calibration was done
clf.save(model_filename)
keras.backend.clear_session()

## Vizualize learned filters
### Raw vizualiation of 1D convolutinal kernel of the first layer --> spatial filters

In [ ]:
# # We first visualize the learned patch embeddings.
# patch_embeddings = clf.layers[0].get_weights()[0]
# weights = patch_embeddings
# # First, apply min-max normalization to the
# # given weights to avoid isotrophic scaling.
# p_min, p_max = weights.min(), weights.max()
# weights = (weights - p_min) / (p_max - p_min)

# # Visualize all the filters.
# num_filters = 10
# plt.figure(figsize=(16, 2))
# idx = 1
# for i in range(num_filters):
#     current_weight = weights[:, :, :, i]
#     if current_weight.shape[-1] == 1:
#         current_weight = current_weight.squeeze()
#     ax = plt.subplot(1, 10, idx)
#     ax.set_xticks([])
#     ax.set_yticks([])
#     plt.plot(current_weight)
#     idx += 1

### Viz of the corresponding topo maps

In [ ]:
# spatialfilts = clf.get_layer("conv2d").get_weights()[0]
# spatialfilts = np.squeeze(spatialfilts)
# spatialfilts = np.swapaxes(spatialfilts, 1, 0)

### Predict on the test set
The predictions are made on windows to regress the code.  
Here we divide the prediction in 10 fold to avoid OOM from the GPU.

In [ ]:
keras.backend.clear_session()

In [ ]:
keras.backend.clear_session()
y_pred = history4.predict(X_test)
y_pred = np.array(y_pred)

In [ ]:
# y_pred_norm = np.array([1 if (y >= 0.5) else 0 for y in y_pred])
# y_test_norm = np.array([0 if y == 0 else 1 for y in Y_test])
y_pred_norm = y_pred
y_test_norm = Y_test

In [ ]:
import sklearn.metrics
import math
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test_norm, y_pred_norm).ravel()
print("True positives:", tp)
print("True negatives:", tn)
print("False positives:", fp)
print("False negatives:", fn)
print("Accuracy:", (tp+tn)/len(y_test_norm))
print("Sensitivity:", sen:=tp/(tp+fn))
print("Precision:", pre:=tp/(tp+fp))
print("Fowlkes-Mallows:", math.sqrt(sen*pre))


True positives: 407
True negatives: 3545
False positives: 335
False negatives: 393
Accuracy: 0.8444444444444444
Sensitivity: 0.50875
Precision: 0.5485175202156334
Fowlkes-Mallows: 0.5282596789550604


### Convert prediction on windows (regressed code) to label prediction

It is offline and synchronous, so we will:
1. First create a `code_buffer` that contains the regressed code on the full epoch (2.2s - the last window)
    - The refresh rate of the EEG device (500 Hz) is faster than the refress rate of the screen (60Hz), so we average predictions (500/60~8 samples) so they correspond to each flip of the screen.
2. Starting from `min_len` (in number of samples), we compute Pearson correalation with the bank of templates code to find the closest one
    - If the most correlated code has a significantely bigger correlation compared to the second one (50% bigger) and the p_value is significative then the trial is classified and we move to the next one
    - If the thresholds are not reached, then we add samples (with a step of 3) to have a longer trial and re-do the computation
    - The thresholds can never been reached in 2.2s, then the trial is not classified.

In [ ]:
# p_best_score = 0
# p_best_ratio = 0
# p_best_acc = 0
# p_best_values = ()

# for p in np.logspace(0,-5, 6):
#     for dr in np.linspace(0.2, 0.8, 7):
#         labels_pred, _, mean_long = make_preds_pvalue(y_pred, codes, min_len=70, sfreq=sfreq, obj_p=p)
#         ratio = len(labels_pred[labels_pred != -1])/len(labels_pred)
#         accuracy = accuracy_score(labels_test[labels_pred != -1], labels_pred[labels_pred != -1])
#         score = ratio*accuracy
#         if ratio >= p_best_ratio:
#             p_best_score = score
#             p_best_acc = accuracy
#             p_best_ratio = ratio
#             p_best_values = (p, dr)
#         print("==========", p, "+", dr,':',ratio,'--',accuracy,'--',score, '--', np.mean(mean_long))
# print("Best score", p_best_score)
# print("Best values", p_best_values)            
        

### Compute accuracy score and accuracy score when a prediction is made (discard not classified trials)

### Other classification method
Same as before but the classification method is different. Instead of thresholds to reach, if when increasing trial lengt a code correllated the most 40 times in a row then the trial is labeled.

In [ ]:
# import importlib
# importlib.reload(_utils)
# from _utils import make_preds_accumul_aggresive, make_preds_pvalue



In [ ]:
acc_best_score=0
acc_best_ratio=0
acc_best_acc=0
acc_best_value = 0
# for minlen in np.linspace(10, 60, 10):
for cons in [15, 20, 25, 30, 35, 40, 45, 50, 55, 60]:
    
    labels_pred_accumul, _, mean_long_accumul = make_preds_accumul_aggresive(
        y_pred_norm, codes, min_len=30, sfreq=60, consecutive=cons, window_size=window_size
    )
    print(labels_pred_accumul)
    print(labels_test)
    ratio = np.round(len(labels_pred_accumul[labels_pred_accumul != -1])/len(labels_pred_accumul), 2)
    accuracy = np.round(accuracy_score(labels_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1]), 2)
    score = np.round(ratio*accuracy,2)
    print("==========", cons, ':', ratio, '--', accuracy,
        '--', score, '--', np.mean(mean_long_accumul))
    if accuracy > acc_best_acc:
        acc_best_score = score
        acc_best_value = cons
        acc_best_acc = accuracy
        acc_best_ratio = ratio

print("Best accuracy", acc_best_acc)
print("Best ratio", acc_best_ratio)
print("Best score", acc_best_score)
print("Best values", acc_best_value)


[0 2 1 3 3 1 2 2 1 0 1 3 3 2 2 0 0 3 1 1 3 2 1 1 2 1 3 2 1 3 2 0 2 2 2 2 2
 1 3 2]
[0 2 1 3 3 2 0 1 1 0 2 3 3 1 2 0 0 3 1 2 3 2 1 0 2 0 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
========== 15 : 1.0 -- 0.68 -- 0.68 -- 0.8083333333333333


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[0 2 1 3 3 2 2 1 1 0 1 3 3 2 2 0 0 3 1 1 3 2 1 0 2 1 3 2 1 3 2 0 0 2 3 2 2
 1 3 2]
[0 2 1 3 3 2 0 1 1 0 2 3 3 1 2 0 0 3 1 2 3 2 1 0 2 0 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
========== 20 : 1.0 -- 0.8 -- 0.8 -- 0.9483333333333335
[0 2 1 3 3 2 2 1 1 0 1 3 3 2 2 0 0 3 1 1 3 2 1 0 2 1 3 2 1 3 2 0 0 2 3 2 2
 1 3 2]
[0 2 1 3 3 2 0 1 1 0 2 3 3 1 2 0 0 3 1 2 3 2 1 0 2 0 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
========== 25 : 1.0 -- 0.8 -- 0.8 -- 1.0316666666666667


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[0 2 1 3 3 1 2 1 1 0 1 3 3 2 2 0 0 3 1 1 3 2 1 0 2 1 3 2 1 3 2 0 0 2 3 2 2
 1 3 2]
[0 2 1 3 3 2 0 1 1 0 2 3 3 1 2 0 0 3 1 2 3 2 1 0 2 0 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
========== 30 : 1.0 -- 0.78 -- 0.78 -- 1.105128205128205


...

[0 2 1 3 3 1 2 1 1 0 1 3 3 2 2 0 0 3 1 2 3 2 1 0 2 1 3 2 1 3 2 0 0 1 3 2 0
 1 3 2]
[0 2 1 3 3 2 0 1 1 0 2 3 3 1 2 0 0 3 1 2 3 2 1 0 2 0 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
========== 50 : 1.0 -- 0.85 -- 0.85 -- 1.4171568627450981


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[0 2 1 3 3 1 2 1 1 0 1 3 3 1 2 0 0 3 1 2 3 2 1 0 2 1 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
[0 2 1 3 3 2 0 1 1 0 2 3 3 1 2 0 0 3 1 2 3 2 1 0 2 0 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
========== 55 : 1.0 -- 0.9 -- 0.9 -- 1.5086021505376344


c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


[0 2 1 3 3 1 2 1 1 0 1 3 3 1 2 0 0 3 1 2 3 2 1 0 2 1 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
[0 2 1 3 3 2 0 1 1 0 2 3 3 1 2 0 0 3 1 2 3 2 1 0 2 0 3 1 1 3 2 0 0 1 3 2 0
 1 3 2]
========== 60 : 1.0 -- 0.9 -- 0.9 -- 1.5811111111111107
Best accuracy 0.9
Best ratio 1.0
Best score 0.9
Best values 55


In [ ]:
accuracy_score(labels_test[labels_pred_accumul!=-1], labels_pred_accumul[labels_pred_accumul!=-1])

0.85

In [ ]:
labels_pred_accumul, aaaaa, mean_long_accumul = make_preds_accumul_aggresive(
        y_pred_norm, codes, min_len=30, sfreq=sfreq, consecutive=15, window_size=window_size
    )

c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
c:\Users\s.velut\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\lib\function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


In [ ]:
len(labels_pred_accumul)


In [ ]:
codes